In [0]:

 ====================== Antriksh’s coforge interview ==================
=================================================================
Q add columns in a df table: name,eng,hindi,math. Sum all the cols(except name)
 //using spark sql
devdf.createOrReplaceTempView("stud")

val odf = spark.sql("select sid,(hindi + math + eng) as total from stud group by sid")
Or
val odf = devdf.select((col("hindi")+col("eng")+col("math")).alias("total"))

Q there are 5 ids (key) , 10 partitions in data. How many files will be there?
ans.	50 files
 val ddf2 = ddf1.repartition(10)
ddf2.write.format("csv").option("path", "partitioned_data2").partitionBy("sid").save
ee
Q Now I want 5 files only (as many number of keys are there)
ans. coalesce()
val ddf2 = ddf1.coalesce(1)
ddf2.write.format("csv").option("path", "partitioned_data3").partitionBy("sid").save


Q In the hive table, I want to remove all duplicate records?
ans. 
create table new table select distinct * from oldtable;
//OR
INSERT OVERWRITE TABLE dup_demo SELECT DISTINCT * FROM dup_demo;


Q empid ename manager_iD
1 ram 4
4 rohit 5
name the manager who is also an employee.
ans. select empid from table where empid in (select manager_id from table);

Q In the hive table, once in a while data is getting updated, retrieve the latest data.
select * from transactions where to_date(dt) in (select max(to_date(dt)) from transactions) ;

Q a table is partitioned on city, 5 city…  new data came with 2 new cities, how will new 2 city partitions be created.
Ans.  

step1) set hive.exec.dynamic.partition.mode=nonstrict;
step2)Create a new table(orders_state3) with the same schema, then insert data into it.
step3)insert into orders_state partition(state) select * from orders_state3;

Q what type of joins are there in spark
Join Strategy Types in spark:
	1. Broadcast Hash Join (broadcast join)
	Small table(df) will go to each partition of big table(df) and will join within each executor.
	2. Shuffle hash join
	Step 1- Shuffling: The data from the Join tables are partitioned based on the Join key. It does shuffle the data across partitions to have the same Join keys of the record assigned to the corresponding partitions.
Step 2- Hash Join: A classic single node Hash Join algorithm is performed for the data on each partition.
	3. Shuffle sort merge join
			Shuffle Phase: Both large tables will be repartitioned as per the Join keys across the partitions in the cluster.
Sort Phase: Sort the data within each partition parallelly.
Merge Phase: Join the sorted and partitioned data. It is a merging of the dataset by iterating over the elements and joining the rows having the same value for the Join keys.
	4. Broadcast Nested Loop Join
	5. Cartesian Join

Q How does SMB work? 
Q. how will you do SMB without buckets in spark.
Q. files are there in multiple folders, you need to create one data frame out of it.
Ans. val input = spark.read.option("header", value = true)
 .csv("C:\\Users\\dev30\\Downloads\\us-500\\data1.csv",
   "C:\\Users\\dev30\\OneDrive\\Documents\\data1.csv")


Q. groupbykey vs reducebykey?
groupByKey:
Syntax:
sparkContext.textFile("hdfs://")
                    .flatMap(line => line.split(" ") )
                    .map(word => (word,1))
                    .groupByKey()
                    .map((x,y) => (x,sum(y)))

groupByKey can cause out of disk problems as data is sent over the network and collected on the reduced workers.

reduceByKey:
Syntax:

sparkContext.textFile("hdfs://")
                    .flatMap(line => line.split(" "))
                    .map(word => (word,1))
                    .reduceByKey((x,y)=> (x+y))

Data is combined at each partition, only one output for one key at each partition to send over the network. reduceByKey required combining all your values into another value with the exact same type.

aggregateByKey:

same as reduceByKey, which takes an initial value.

3 parameters as input i. initial value ii. Combiner logic iii. sequence op logic


Example:
val keysWithValuesList = Array("foo=A", "foo=A", "foo=A", "foo=A", "foo=B", "bar=C", "bar=D", "bar=D")
    val data = sc.parallelize(keysWithValuesList)
    //Create key value pairs
    val kv = data.map(_.split("=")).map(v => (v(0), v(1))).cache()
    val initialCount = 0;
    val addToCounts = (n: Int, v: String) => n + 1
    val sumPartitionCounts = (p1: Int, p2: Int) => p1 + p2
    val countByKey = kv.aggregateByKey(initialCount)(addToCounts, sumPartitionCounts)

output: Aggregate By Key sum Results bar -> 3 foo -> 5

combineByKey:

3 parameters as input

    Initial value: unlike aggregateByKey, need not pass constant always, we can pass a function that will return a new value.
    merging function
    combine function

Example:

val result = rdd.combineByKey(
                        (v) => (v,1),
                        ( (acc:(Int,Int),v) => acc._1 +v , acc._2 +1 ) ,
                        ( acc1:(Int,Int),acc2:(Int,Int) => (acc1._1+acc2._1) , (acc1._2+acc2._2)) 
                        ).map( { case (k,v) => (k,v._1/v._2.toDouble) })
        result.collect.foreach(println)

Q. where do we need to use groupkey?

Q group by key partition? 200, key 
K1ey  number
1	20
1	30
2	60
2 	 80
groupbyKey 1,(20 + 30)
2 , (60+80)

reduce ByKey (1,60)

Q for each department, give emp ids of 5th highest salary.
Ans. 
select dept_id, dense_rank() over(partition by dept_id order by salary desc) as rnk from table where rnk=5;

Q.How Serialization helps in optimization.
●	Serialization plays an important role in the performance for any distributed application. By default, Spark uses Java serializer.
●	Spark can also use another serializer called ‘Kryo’ serializer for better performance.
●	Kryo serializer is in compact binary format and offers processing 10x faster than Java serializer.
●	To set the serializer properties:
conf.set(“spark.serializer”, “org.apache.spark.serializer.KryoSerializer”

Q. physical plan vs logical plan.
Logical Plan just depicts what I expect as output after applying a series of transformations like join, filter, where, groupBy, etc clause on a particular table.
Physical Plan is responsible for deciding the type of join, the sequence of the execution of filter, where, groupBy clause, etc.
This is how SPARK SQL works internally!!!

Q. How does a physical plan work?

Q. You have data in 4 partitions, what will happen when you will use mapPartition, and when you will use map. How is it going to process?
	Map will iterate row by row while map partition will iterate partition by partition

Q. How will you see the logical plan and physical plan?
Ans  using Explain or logs

========================================================================
Dev’s Unify Tech R1
========================================================================
Q1- All occurrences of second highest element in array
object practice {
  def main ( args: Array[String]){
    val l1 = List(10,9,11,7,8,6,5,12,12,11,11,4)
    var h1:Int = l1.min
 
    var h2:Int = h1
    def secMax(l:List[Int]):Int =
    {
      for(x <- l)
      {
        if(x>h1)
        {
          h2=h1
          h1=x
        }
        else if(x>h2 && x!=h1)
        {
          h2=x
        }
      }
     h2
    }
    
    var h4:Int = secMax(l1)
    println(l1.count(x=>x==h4))
    
  }
}


Ques 2: - sort-merge-join vs broadcast join 

Broadcast Hash Join
	Send the small df to each partition node.

 Shuffle hash join
	Shuffle is involved in big df to get similar keys in the same partition. Then it sends the small df to each partitioned node.
Shuffle sort-merge join
	Each partition has sorted keys, similarly sorted keys from other tables will be shuffled in the same partition.


Q3 - create a file and write data on it 
   Explained touch and cat command
Q4- how would you create a spark session 
Val spark = sparkSession.builder()
      .appName(“xyz”)
      .master(“local[*]).getOrCreate

Q- 5 - 
spark-shell --master yarn --numexecuter 20 --executor-memory 21G --executors-core 5 

spark-submit --class class-name /sapark_pro.jar   --master yarn --numexecuter 20 --executor-memory 21G --executors-core 5 
Q- 6- Name of the students who scored second highest marks
mark
id,name,marks
1,R1,78
2,R2,75
3,R3,78
4,R4,75

Ans - R2
R4
select name, dence_rank() over(order by marks) as rnk form mark where rnk=2;

========================================================================
Dev’s Unify Tech R2 
========================================================================

Q1- //List - [10,9,11,7,8,6,5,12,4] - Second largest Number

object HelloWorld {
 def main(args: Array[String]) = println("Hello Scala!")

 val l = List(10, 9, 11, 7, 8, 6, 5, 12, 12, 4)
 var first = 0
 var Second = 0

 def compare(v: Int) = {
   //println(v)
   if (v > first) {
     Second = first
     first = v
   }
   else if (v > Second && v != first) {
     Second = v
   }
 }

 val greater = l.map(x => compare(x))
 println(Second)
}

Q-2 given two files sum up both the file column wise ?
/*
File1.txt
=========
1 2 3 1
1 2 3 2
2 2 2 1

File2.txt
=========
2 3 1 1
3 1 2 1
1 1 1 1

Col1 = 1+1+2+2+3+1 = 10
Col2 = 11
Col3 = 12
Col4 = 7

import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import scala.io.StdIn

object interview_code_spark extends App {
 Logger.getLogger("org").setLevel(Level.ERROR)
 val sc = new SparkContext("local[*]", "word_count")

 val input1 = sc.textFile("D:\\file1.txt")
 val input2 = sc.textFile("D:\\file2.txt")

 val combined = input1.union(input2)
 val comb_split = combined.map(x => x.split(" "))

 val comb_int = comb_split.map(x => x.map(e => e.toInt))
 val resultant = comb_int.reduce((xx, yy) => xx.zip(yy).map { case (a, b) => a + b })

 resultant.foreach(println)

 StdIn.readInt()

}

Q. window function in spark using dataframe

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

    val windowSpec = Window.partitionBy("name").orderBy("marks")
     
     df1.withColumn("row_number",row_number.over(windowSpec)).show()
========================================================================
	Dev’s Mind tree 
========================================================================

Q1- how to read csv file having , separated fields and also , separated values in columns?? 
Approach ⇒ dataframe, RDD + regular expression

(in csv column values are inside brackets "", if we don't have "", we will lose the data )

Sno,name,marks
1,dev,kumar,70
2,ant,kethwas,90
3,raj,kumar,100
4,jay,verma,88

1,”dev,kumar”,70
2,”ant,kethwas”,90
3,”raj,kumar”,100
4,”jay,verma”,88

val Input = StdIn.readLine()
Val Input2 = sc.parallelize(input)

Val input3 = input2.map(x=>{
Val a = x.split(“,”)
(a(0),a(1)+a(2),a(3)) 
})


Q2- write a program to create an array of elements of the first array that are not in the second array ?? //

import scala.io.StdIn._

object A_minus_B_array {

 def main(args: Array[String]): Unit = {

   val a = readLine.split(" ").map(_.toInt)
   val b = readLine.split(" ").map(_.toInt)

   val ans = findMissing(a, b)
   ans.foreach(println)
 }

 def findMissing(a: Array[Int], b: Array[Int]): List[Int] = {
   var list = List[Int]()
   for (elem <- a if !b.contains(elem))
     list = list :+ elem
   list
 }
}

Q3- find the employee details having the third largest salary?

Q4- replace a column having values like 10k, 30k to 10000, 30000 and so on??
Ans:

System.setProperty("hadoop.home.dir", "C:\\HADOOP_HOME\\")
Logger.getLogger("org").setLevel(Level.ERROR)

val spark = SparkSession.builder
 .appName("first app")
 .master("local[*]")
 .getOrCreate()

val devdf = spark.read
 .option("header", true)
 .csv("dev1.csv")

devdf.createOrReplaceTempView("emp")

val odf = spark.sql("select emp_id,emp_name,concat(substr(salary,0,length(salary)-1),'000') as salary from emp")

val odf2 = odf.withColumn("salary", col("salary").cast("Int"))

odf2.printSchema()

val partitionWindow = Window.orderBy(col("salary").desc)

val partitionWindowRank = rank().over(partitionWindow)

val odf3 = odf2.select(col("*"), partitionWindowRank as "dense_rank").where(col("dense_rank") === 2)

odf3.show
odf2.show


Q5- repartition and coalesce ?? (when, how)
 

Q6- how will you save the same data in json format?
input2.write.format("json").option("path","C:\\Users\\dev30\\Downloads\\us-500\\us-500.json").save

========================================================================= 
  Dheeraj = Mindtree
========================================================================

Q7 - What is broadcast in spark

Q8. Dataframe vs Dataset

Q9. Bucket in hive

Q10. How to read from kafka and offset

kafka-console-consumer --topic example-topic --bootstrap-server broker:9092 \
 --property print.key=true \
 --property key.separator="-" \
 --partition 1 \
 --offset 6

Q11. Write a code to read parquet file(product files and orders files) using spark and show all the data available in that file, status = sold 

Q12. What is Data Lineage (RDD)

Q13.  Create IPL schedule like each team play at least 2 match with every team

KKR	
DC	
RCB	
CSK	
MI	
Pune	
KP	
SH	

Q14. Find the duplicate from the given dataset using dataframe
Q15. How to handle duplicate reading in kafka
=======================================================================  
  Dheeraj = NH Health
=====================================================================
1.	What is difference b/w reparations and coalesce
2.	What is salting and the cause of using salting
3.	What happen if file size is small
4.	What is memory out of exception in spark
5.	How to deploy spark code and monitor and how do you test the code
6.	How stages creates in spark and when it gets created
7.	Difference between persist and cache and broadcast join
8.	What all tables available in your project and how you join those tables
9.	Where is your big data setup, cloud or on-premises
10.	Given one table and find the the employee total working hours using only one column which is “timestamp”
11.	What is DAG
12.	What is difference b/w val or var
13.	Write spark code to read csv file using scala
14.	How to join two table and get only the left table matching information from right table
15.	Write the syntax for repartition and coalesce
16.	Scenario given like
	1 flatmap
2. Map
3. Persist
4. reduceByKey
5. groupByKey
6. Df.count
17.	Tell me the flow of execution and number of stages 
18.	What is EMR and how you deploy spark code using EMR and also how to create EMR cluster and SSH to your machine
19.	Where you store hive table data in hdfs or cloud and can hdfs is in cloud also ?(tricky question)
20.	When stages gets create on spark UI
21.	Tell me the complete project flow in organization project like from development to deploy spark code and how you monitor

========================================================================
Dev’s coforge
========================================================================

1.	size of the hdfs files 
--> hadoop fs -du -s -h  /user

2.	Find number of nulls in a dataframe column
--> df.filter(df(colName).isNull || df(colName) === "" || df(colName).isNaN).count()


3.	Hcatalog in hive 
Basically, a table as well as a storage management layer for Hadoop is what we call HCatalog. Its main function is that it enables users with different data processing tools, for example, Pig, MapReduce to make the read and write data easily on the grid.

4.	change a managed table into  external table
	alter table table_name SET TBLPROPERTIES('EXTERNAL'='TRUE');

5.	sqoop command to import data from S3 
	sqoop import -Dfs.s3a.access.key=$ACCES_KEY -Dfs.s3a.secret.key=$SECRET_KEY
--connect $CONN --username $USER --password $PWD --table $TABLENAME --target-dir s3a://example-bucket/target-directory

6.	read multiple csvs 
→ val input = spark.read.option("header", value = true)
 .csv("C:\\Users\\dev30\\Downloads\\us-500\\data1.csv",
   "C:\\Users\\dev30\\OneDrive\\Documents\\data1.csv")

→ spark.read.format("csv").option("header", "true").load("../Downloads/*.csv") spark.read.option("header", "true").csv("../Downloads/*.csv")

7.	hive vs impala 
Cloudera Impala being a native query language, avoids startup overhead which is commonly seen in MapReduce/Tez based jobs (MapReduce programs take time before all nodes are running at full capacity). In Hive, every query has this problem of “cold start” whereas Impala daemon processes are started at boot time itself, always being ready to process a query.
Hadoop reuses JVM instances to reduce startup overhead partially but introduces another problem when large haps are in use. Cloudera benchmarks have 384 GB memory which is a big challenge for the garbage collector of the reused JVM instances.
MapReduce materializes all intermediate results, which enables better scalability and fault tolerance (while slowing down data processing). Impala streams intermediate results between executors (trading off scalability).
Hive generates query expressions at compile time whereas Impala does runtime code generation for “big loops”.
Apache Hive might not be ideal for interactive computing whereas Impala is meant for interactive computing.
Hive is batch based Hadoop MapReduce whereas Impala is more like an MPP database.
Hive supports complex types but Impala does not.
Apache Hive is fault tolerant whereas Impala does not support fault tolerance. When a hive query is run and if the DataNode goes down while the query is being executed, the output of the query will be produced as Hive is fault tolerant. However, that is not the case with Impala. If a query execution fails in Impala it has to be started all over again.

8.	how to fix HDFS corrupt file
hdfs fsck /
9.	Why  spark, hive itself can do this?
 
10.	How to change data type of column in a hive table?
→ ALTER TABLE tableA CHANGE ts ts BIGINT AFTER id;
11.	Explode in a hive ?? 
The LATERAL VIEW statement is used with user-defined table generating functions such as EXPLODE() to flatten the map or array type of a column.The explode function can be used on both ARRAY and MAP with LATERAL VIEW.   
Explode function:
The explode function explodes an array to multiple rows. Returns a row-set with a single column (col), one row for each element from the array.  
Lateral View :
Lateral views explode the array data into multiple rows. In other words, lateral view expands the array into rows. 
select std_id,stud_name,location,courses from std_course_details LATERAL VIEW explode(course) courses_list as courses;

========================================================================
Dheeraj = capgemini
========================================================================

1.	Running spark jobs 
2.	Accumulator vs broadcast variable 
Result - got 
========================================================================
antriksh= capgemini
========================================================================


Authors table
authorname	bookname	

Rabindra	b1
james		b2
robin		b3
Satyajeet	b4
Noha		b5
Nolan		b6
Elon		b7
Rabindra	b8	Books table
bookname soldcopies

b1	3
b2	25
b3	3
b4	10
B5 	0
b6	12
B7	8
B8	10

Q. top 3 authors who sold most books using the above 2 tables.
select authorname, sum(soldcopies) s from 
authors a join 
books b on a.bookname=b.bookname 
group by authorname 
order by s desc limit 3;

Q stream vs batch

Q types of files used in spark
  ORC, Parquert , CSV , text ,sequence 
Q you have a csv file of 70 lac records (big). you need to upload it to the database. but it is taking 12 hours right now, do it in less time.

Q how to update column data type in hive
ALTER TABLE table_name CHANGE column_name column_name new_datatype;

Q types of time data type in hive

Q we are uploading data into the database from a csv file, it is taking 14 hours. Data is big. What should we do to reduce processing time from 14 hours so that it takes less time?

========================================================================
					Antriksh’s BCG
========================================================================

https://towardsdatascience.com/strategies-of-spark-join-c0e7b4572bcf

Q1 Tell me about yourself
Q2 can you explain working of your project from business point of view
Q3 size of cluster, node size, cores, 
Q4 Spark Architecture
Q5 Spark session vs spark context
Ans. In spark 2, there is a universal entry point for the spark engine.
Spark Context:
Prior to Spark 2.0.0 sparkContext was used as a channel to access all spark functionality.
The spark driver program uses spark context to connect to the cluster through a resource manager (YARN orMesos..).
sparkConf is required to create the spark context object, which stores configuration parameters like appName (to identify your spark driver), application, number of cores and memory size of the executor running on the worker node.

In order to use APIs of SQL, HIVE, and Streaming, separate contexts need to be created.

Example:
creating sparkConf :

val conf = new SparkConf().setAppName(“RetailDataAnalysis”).setMaster(“spark://master:7077”).set(“spark.executor.memory”, “2g”)

creation of sparkContext:
val sc = new SparkContext(conf)

Spark Session:

SPARK 2.0.0 onwards, SparkSession provides a single point of entry to interact with underlying Spark functionality and
allows programming Spark with DataFrame and Dataset APIs. All the functionality available with sparkContext are also available in sparkSession.

In order to use APIs of SQL, HIVE, and Streaming, no need to create separate contexts as sparkSession includes all the APIs.

Once the SparkSession is instantiated, we can configure Spark’s run-time config properties.

Example:

Creating Spark session:
val spark = SparkSession
.builder
.appName(“WorldBankIndex”)
.getOrCreate()

Configuring properties:
spark.conf.set(“spark.sql.shuffle.partitions”, 6)
spark.conf.set(“spark.executor.memory”, “2g”)

Spark 2.0.0 onwards, it is better to use sparkSession as it provides access to all the spark Functionalities that sparkContext does. Also, it provides APIs to work on DataFrames and Datasets.

Q6 When will you get an OOM error in the driver machine?
Ans. 
1) collect
	2) broadcast join

Q7 when you write your dataframe, we get an output file in multiple partitions. Can you get your csv output file into one single file?
Ans. 
df.coalesce(1).write.csv("address")
df.repartition(1).write.csv("address")

Q8 you are reading data in two dataframes, applied join on your dataframe and saved it. How many stages will be created?

Q9 property to set partitions constant throughout application

Q10 Where do you use SparkSQL?
Spark SQL is a Spark module used for structured data processing. DataFrames represent a distributed collection of data, in which data is organized into columns that are named. DataFrames provide a domain-specific language that can be used for structured data manipulation in Java, Scala, and Python.

Q11 you have 6 node clusters, 64 gb, 16 core.
Ans. how will you decide the number of exec, cores, memory.

63 gb, 15 core
5 core, 21 gb
3 executors per node.

Thin exec: 1 core= 1 exec. No parallelism
Fat exec: 15 core = 1 exec. Throughput suffers

Q spark memory distribution
 
Q is “persist” an action or transformation?
Though cache() or persist() is just another function on RDD which marks RDD to be cached or persisted. The first time an RDD is evaluated as a consequence of an action, it will be persisted/cached. So, cache() or persist() is neither an action nor a transformation

Q When will you use coalesce and when will you use repartition?
coalesce uses existing partitions to minimize the amount of data that's shuffled. repartition creates new partitions and does a full shuffle. coalesce results in partitions with different amounts of data (sometimes partitions that have much different sizes) and repartition results in roughly equal sized partitions.

Q tell any instance when you encountered OOMs in your work.
Ans. I was aggregating on the basis of customer segment. For example if data is kept in 8 partitions, then let’s say there are 2 customer segments then data will be aggregated or shuffled to these two partitions and they are not able to accommodate. 

Q where do we use spark context?

Main entry point for Spark functionality. A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster. Note: Only one SparkContext should be active per JVM.

Q you have one string “my name is Antriksh” replace your name with xyz.
val str = "my name is antsant"
val s= str.replace("antsant","dev")

Q “geeksforgeeks” prints only duplicate characters.
Ans.
val str2 = "ParallelCollectionRDD"
val s = str2.foldLeft(Map[Char, Int]()) {
      case (map, char) =>
        val oldCount: Int = map.getOrElse(char, 0)
        map.+(char -> (oldCount + 1))
    }
.filter(x => x._2 >=2).map(x => x._1)

Q Traits in scala?
Traits are like interfaces in Java. But they are more powerful than the interface in Java because in the traits you are allowed to implement the members. Traits can have methods(both abstract and non-abstract), and fields as its members.
Traits are created using trait keywords.
// Scala program to illustrate how to
// create traits

// Trait
trait MyTrait
{
	def pet
	def pet_color
}

// MyClass inherits trait
class MyClass extends MyTrait
{
	
	// Implementation of methods of MyTrait
	def pet()
	{
		println("Pet: Dog")
	}
	
	def pet_color()
	{
		println("Pet_color: White")
	}
	
	// Class method
	def pet_name()
	{
		println("Pet_name: Dollar")
	}
}

object Main
{
	
	// Main method
	def main(args: Array[String])
	{
		val obj = new MyClass();
		obj.pet();
		obj.pet_color();
		obj.pet_name();
	}
}
In Scala, we are allowed to implement the method(only abstract methods) in traits. If a trait contains method implementation, then the class which extends this trait need not implement the method which is already implemented in a trait. As shown in the below example.
Example:
// Scala program to illustrate the concept of
// abstract and non-abstract method in Traits
  
// Trait with abstract and non-abstract methods
trait MyTrait
{
    // Abstract method 
    def greeting
  
    // Non-abstract method
    def tutorial
    {
        println("This is a tutorial" + 
                "of Traits in Scala")
    }
}
  
  
// MyClass inherits trait
class MyClass extends MyTrait
{
      
    // Implementation of abstract method
    // No need to implement a non-abstract 
    // method because it already implemented
    def greeting()
    {
        println("Welcome to GeeksfoGeeks")
    }
} 
  
object Main
{
    // Main method
    def main(args: Array[String]) 
    {
        val obj = new MyClass();
        obj.greeting
        obj.tutorial
    }
}

Output:
Welcome to GeeksfoGeeks
This is a tutorial of Traits in Scala

Traits do not contain constructor parameters.
When a class inherits one trait, then use extends keyword.
Syntax:
class Class_Name extends Trait_Name{
// Code..
}
When a class inherits multiple traits then use extends keyword before the first trait and after that use with keyword before other traits. As shown in the below example.
Syntax:
class Class_Name extends Trait_Name1 with Trait_Name2 with Trait_Name3{
// Code..
}
Example:
// Scala program to illustrate how
// a class inherits multiple traits
  
// Trait 1
trait MyTrait1
{   
    // Abstract method 
    def greeting 
}
  
//Trait 2
trait MyTrait2
{ 
    // Non-abstract method
    def tutorial
    {
        println("This is a tutorial" + 
               "of Traits in Scala")
    }
}
  
// MyClass inherits multiple traits
class MyClass extends MyTrait1 with MyTrait2
{
      
    // Implementation of abstract method
    def greeting()
    {
        println("Welcome to GeeksfoGeeks")
    }
} 
  
object Main 
{
    // Main method
    def main(args: Array[String]) 
    {
        val obj = new MyClass();
        obj.greeting
        obj.tutorial
    }
}

Output:
Welcome to GeeksfoGeeks
This is a tutorial of Traits in Scala

An abstract class can also inherit traits by using extends keyword.
Syntax:
abstract class Class_name extends Trait_Name{
// code..
}
In Scala, one trait can inherit another trait by using an extends   keyword.
Syntax:
trait Trait_Name1 extends Trait_Name2{
// Code..
}
Traits support multiple inheritance.
In Scala, a class can inherit both normal classes or abstract classes and traits by using extends keyword before the class name and with the keyword before the trait’s name.
Syntax:
class Class_Name1 extends Class_Name2 with Trait_Name{
// Code..
}
In Traits, abstract fields are those fields containing initial value and concrete fields are those fields which contain the initial value. we are allowed to override them in the class which extends the trait. If a field is declared using the var keyword, then there is no need to write an override keyword when we override them. And if a field is declared using the val keyword, then you must write the override keyword when we override them.
Example:
// Scala program to illustrate 
// concrete and abstract fields in traits
  
trait MyTrait
{
      
    // Abstract field
    var value: Int 
      
    // Concrete field
    var Height = 10
    val Width = 30
}
 
class MyClass extends MyTrait
{
      
    // Overriding MyTrait's fields
    var value = 12
    Height = 40
    override val Width = 10
      
    // Method to display the fields
    def Display()
    {
        printf("Value:%d", value);
        printf("\nHeight:%d" ,Height);
        printf("\nWidth:%d", Width);
    }
}
  
object Main
{
      
    // Main method
    def main(args: Array[String])
    {
        val obj = new MyClass();
        obj.Display();
    }
}

Output:
Value:12
Height:40
Width:10
We can also add traits to an object instance. Or in other words, We can directly add a trait in the object of a class without inheriting that trait into the class. We can add a trait in the object instance by using a keyword.
Syntax:
val object_name = new Class_name with Trait_Name;
Example:
// Scala program to illustrate how 
// to add a trait to an object instance 
   
class MyClass{}
trait MyTrait
{
    println("Welcome to MyTrait");
}
object Main 
{
      
    // Main method
    def main(args: Array[String])
    {
          
        // Here MyTrait is added to the 
        // object instance of MyClass
        val obj = new MyClass with MyTrait;
    }
}

Q tail recursion in scala?
A recursive function is tail recursive when a recursive call is the last thing executed by the function.
============================ capgemini L2 Antriksh=========================

1.	You are given some data, you need to use few operations against that data. How would you use HIVE or Spark?? (when will you use Hive and when Spark)

2.	Spark vs MapReduce ??

3.	Spark framework ? How is it different from Mapreduce?

4.	How would you say spark is faster than Mapreduce? 

5.	You have given a list say l1 = [1,4,3,2,4,4,4,2,3,5] . write a function in scala to print the frequency of duplicate numbers ??
6.	How do you save the spark code you use? In what extension?
a.	.scala
7.	How do you execute your spark job? In what format is it in spark?
8.	How would you create a jar file
9.	What command would you write to submit your spark job?how would you give resources? How dynamic resource allocation would work?
10.	Name some transformations and actions
11.	Partitioning vs bucketing

========================== Kumulus Pvt. Ltd. c============================

1.	Given one data table where you have to choose only year from the date column
Ans: select date_format(“date”, “M”) as year from datetable
2.	6 Node and 64 ram, 64 core, how many executor will be there
Ans: explain same thing which is taught in the course
3.	Persist and cache
4.	Read Json file, where data is stored as key, value pair, Write spark code to pick only key from the file
Ans: i) using RDD
II) using dataframe by creating table with manual schema
5.	How to read data from kafka using spark
6.	What is accumulator in spark
Accumulators are variables that are only “added” to through an associative and commutative operation and can therefore be efficiently supported in parallel. They can be used to implement counters (as in MapReduce) or sums. Spark natively supports accumulators of numeric types, and programmers can add support for new types.
Result - got 
===================== Maveric - Dheeraj ===============================

1. I need to bring the data to HDFS using sqoop. Data is changing 3 times a day, so what would be my design approach?
2. write spark code for filtering data from HIVE/HDFS (Sal>10000) and store in another HIVE table with id, name, level, age and salary. Use Spark Core functions.
3. write spark code for finding employees whose level changed from 2 to 3 in the last one month, and increase their amount by 15%. Update existing table.

======================= All State - Antriksh ==============================

Q project overview?

Q challenges you face on a regular basis?
Ans. We often need to aggregate data at customer segments and we may end up getting oom error. We use salting to solve it.

Q What would you do if a cluster fails?
Ans. We never encounter such problems. I think administrators will look into these types of failure.

Q What if a node will fail while your job is executing?

Q shell script? Find a file which was created/modified before a certain date? Can you do it in scala?

ls -lt  | grep 'Apr 17'

find . -type f -ls |grep 'filename'


Q Is it still important to use rdd code? If yes, then when?
There are many instances where we need to use lower level constructs 
1.	When we read data from unstructured files and need to apply lower level transformations 
2.	To check number of partition of DF

Q How do you see dag for dataframe?
 Spark web UI

Q What is dag?
In Airflow, a DAG – or a Directed Acyclic Graph – is a collection of all the tasks you want to run, organized in a way that reflects their relationships and dependencies.
A DAG is defined in a Python script, which represents the DAGs structure (tasks and their dependencies) as code.
Q How do you read from a kafka stream?
# Start ZooKeeper.  Run this command in its own terminal.
  ./bin/zookeeper-server-start ./etc/kafka/zookeeper.properties

# Start Kafka.  Run this command in its own terminal
  ./bin/kafka-server-start ./etc/kafka/server.properties

# Create the input topic
  ./bin/kafka-topics --create \
          --bootstrap-server localhost:9092 \
          --replication-factor 1 \
          --partitions 1 \
          --topic streams-plaintext-input

# Create the output topic
  ./bin/kafka-topics --create \
          --bootstrap-server localhost:9092 \
          --replication-factor 1 \
          --partitions 1 \
          --topic streams-wordcount-output

Next, we generate some input data and store it in a local file at

 /tmp/file-input.txt:

echo -e "all streams lead to kafka\nhello kafka streams\njoin kafka summit" > /tmp/file-input.txt
Lastly, we send this input data to the input topic:
cat /tmp/file-input.txt | ./bin/kafka-console-producer --broker-list localhost:9092 --topic streams-plaintext-input
# Run the WordCount demo application.
# The application writes its results to a Kafka output topic -- there won't be any STDOUT output in your console.
# You can safely ignore any WARN log messages.
  ./bin/kafka-run-class org.apache.kafka.streams.examples.wordcount.WordCountDemo

./bin/kafka-console-consumer --bootstrap-server localhost:9092 \
        --topic streams-wordcount-output \
        --from-beginning \
        --formatter kafka.tools.DefaultMessageFormatter \
        --property print.key=true \
        --property key.deserializer=org.apache.kafka.common.serialization.StringDeserializer \
        --property value.deserializer=org.apache.kafka.common.serialization.LongDeserializer
======================================================================
Q 
Input:

table1: 
userid| Ip address 
1|190.8.18.7
2|190.8.9.7

tabl2:
userid| address| username
1|AAAA|u
2|BBBB|A
3|CCCC|C
4
5
6
...
3,6,9,12

output
Userid|ipaddress|username|address

Get the output. Do it in sql, then do it using df.
Ans.
select t1.userid,t1.ipaddress,t2.username,t2.address
	table1 t1 join table2 t2
	on t1.userid=t2.userid;

df1.join(df2,df1.col("userid")===df2.col("userid"),inner)

Q in above table, get records which are divisible by 3 (ex 3,6,9,12,15,....)
Ans

rdd.filter(x=>x._1%3==0)
	or
df.where(col(“id”)%3==0)

Q you have data from chat logs. write code to find all the occurrences of “utkarsh”.

1, "hi utkarsh how are you" 
2, "what's up"

Ans.

var input = textFile("some location")

var flatInp = input.flatmap(x=>x._2.split(" "))

var findNm = flatInp.filter(x=>x=="utkarsh")

var output = findNm.count

//can we use count instead of reducebykey? As the code is in pure scala

Q on what factor parallelism in your spark code depends on?

Core,partitions,key

Q 
job1 SHTP
job2 
Job3 

Job4

Schedule a job, such that if job1 fails then execute job4, if it doesn’t then execute job2 and then 3
Ans.

Q
Input Dataframe:
Date, State, ActiveCount
2021/01/01, Karnataka, 4000
2021/01/01, Delhi, 3500
2021/01/01, Assam, 400
2021/01/02, Karnataka, 300
2021/01/02, Delhi, 3300
2021/01/02, Assam, 420

    Output Dataframe:
Date, State, ActiveCount, Threshold
2021/01/01, Karnataka, 4000, Y
2021/01/01, Delhi, 3500, Y
2021/01/01, Assam, 400, N
2021/01/02, Karnataka, 300, N
2021/01/02, Delhi, 3300, Y
2021/01/02, Assam, 420, N

If activeCount>1000 then put threshold as Y else N

Ans.

def thresh(activeCount:Int):Char = 
{
	if(activeCount>1000)
	{'y'}
	else
	{'N'}
}


udf("thresh",thresh(_:Int):Char)

InputDf.withColumn(“threshold”,"thresh(activeCount)")
	Or

val newDf = df.withColumn("thresh", when(col(“activeCount”) >=1000, “Y”).otherwise(“N”) 

//inputDf.withColumn(“threshold”,col(“”))


Q write binary search in scala
Ans.
 
var l1 = List(1,4,5,7,8,12,33)
var s = 5
var n = l1.size
def bisearch(l1,a,n,s)
{
	if(l1(n/2)==s)
	{l1(n/2)}
	else if(l1(n/2)<s)
	{
		bisearch(l1,0,n/2,s)
	}
	else if(l1(i)>l1(n/2)
	{
		bisearch(l1,n/2,n,s)
	}
}

Q
userid| Ip address 
1|190.8.18.7
2|190.8.9.7

userid| address| username
1|AAAA|u
2|BBBB|A
3|CCCC|C

Ans.

Antijoin

========================= Media.net coding test ===========================

Q 
event	startDate	EndDate
A1	10-07-21	11-07-21
A2	11-07-21	12-07-21
B	15-07-21	16-07-21
D	01-08-21	02-08-21

o/p
event	startDate	EndDate
A1A2	10-07-21	12-07-21
B	15-07-21	16-07-21
D	01-08-21	02-08-21

Explanation: A1 & A2 are two different events in the input table. You need to write a sql query to combine all the events which are falling on consecutive dates.

Q
Var input1= List(1,2,3,5,7,9)	//yes
Var input2 = List(2,-5,3,10,11)	//no

Write a code (preferably in scala) to find if a sub-array has sum with it’s element as zero.
Ex: in above l2 we have 2+3-5=0

o/p should be 1 if it exists and 0 if it doesn’t

Q
Movie_actors
Mid	aid	mcid

Movies
Movie_title	mid


Write a query to print movie_title where the actors have worked in more than one movie.
Ans.
Select m.movie_title 
from movies m join movie_actos ma
On ma.mid=m.mid
where m.aid in (select m.aid from movie_actos group by aid having count(*)>1);
=======================================================================
Dev’s legato
======================================================================

Q1. Joins in SQL 
 No of records after applying inner, left outer and right outer join

Q2 .-  row num, rank and dense_rank ??

Q3. - unix shell script to read a text file “|” delimited , and print the third column?
	Explained in spark scala
Q.4 - one file query.hql  is given with a simple query 

“Select * from db.tablename where id = some_number
Modify the query.hql to accept db , tbl name and id from the hive  CLI while executing the file.
Hint = Hive config

hive -f filename.hql

beeline -u "jdbc:hive2://master01:2181,master02:2181,master03:2181/;serviceDiscoveryMode=zooKeeper;zooKeeperNamespace=hiveserver2" -f file.hql --hivevar HDFSDIR=/tmp/folder

Q5. -  analytical functions in Hive   

 ========================Cognizant -Dheeraj===============================

1.	Remove duplicate records from hive
INSERT OVERWRITE TABLE dup_demo SELECT DISTINCT * FROM dup_demo;

2.	Word count program using df/ spark sql

3.	What happens when cache memory full 

4.	Spark Architecture

5.	Case class

6.	DAG

7.	Lineage
=====================================================================
DUNNHUMBY Dev
===================================================================

Hive - partition , bucketing (scenario based)

Spark - repartition , coalesce (concept) ,What is broadcast in spark
Scenario based question - cache, persists

Sql queries == PostgreSQL exercises (pgexercises.com)
PostgreSQL exercises (pgexercises.com)

Store 	| Recpt_num    | date 		| prod_code 
------------------------------------------------
S01     | R0105        |  2019-01-05|  P12321
S01     | R0158        |  2019-01-09|  P12GBY
S02     | R487          |  2019-05-12|  P234
S02     | R0896        |  2020-01-11|  P9087


Write trasformation to create col tran_code using logic "tran_code => combine store, recpt_num, date ('-' removed)  seperated by '_'. Example value S01_R0105_20190105"
df1.show(false)

val df2 = df1.withColum("tran_code", concat(col("Store"),"_",col("Recpt_num","_",col("date").replace('-',''))
                           
              //col._*.mkString("-")  //concat_ws()
val dfResults = dfSource.select(concat_ws(",",dfSource.columns.map(c => col(c)): _*))

                                            
l2 = [1,None,2,3,None, None,4,5,None]
out = [1,1,2,3,3,3,4,5,5]
                                          var last_known = l2(0)
                                            for( i <- 1 until l2.lenght)
                                            {									
                                              if(l2(i) == None)
                                              {
                                                 l2(i) =  last_known 
                                              }
                                              
                                              else 
                                               last_known = l2(i)
                                            }

Result - got 
==============================================================
======================= Hashed by Deloitte Antriksh) ================

Design classes and database for BookMyShow portal
---------------------------------------------------------------------------------
*Requirements:*
- Users should login to start using the portal.
- The portal should list down the different cities where the theatres are located.
- Once the user selects the city it should display the movies released in that particular city to that user.
- Once the user selects the movie, the portal should display the cinemas running that movie and the available shows.
- Users should be able to select the show at a particular theater.
- The portal should display the seating arrangement of the cinema hall to the user.
- Users should be able to select multiple seats according to their choice.
- Once seats are selected the user will book the ticket.

Users 
User_id varchar
Password password 

City 
City_id int
City name varchar 

Theaters 
Theater_id 
Theater_name 
Th_city
Location 
Available_sheets [100,60,40,5,3]

show
Show_id
M_start_time
M_end_time


Movies 
Movie_id
Movie_name

Bookings
B_id
User_id 
Theater_id
show_id
Movie_id
seats

B01 u01 t23 s1 “avengers” 60,40

- The portal should list down the different cities where the theatres are located.
Select distinct(city) from theatres;

- Once the user selects the city it should display the movies released in that particular city to that user.

Q Nosql vs sql datbase
==================================================================
++++++++++++++++++++ Dev’s paypal ++++++++++++++++++++++++++

1 = What is a tree?
Advantages of tree DS?

Complexity 

2 - Find the index of a number in a given array?
 
Told him = zipWithIndex method
 Linear  search 

Tell me more Optimize way 
If the array is sorted and  rotated 

== binary search => could not implement for the rotated array
Explain = arr.indexOf(element)

3 =  call by value and reference 

==============================================================================================dev surprize interview with impetus ==================
1.	Working of HDFS? When you put a file from local into HDFS how does it happen?
We can first move the file from our local to the edge node’s local using winscp. 
Once we have our file in the edge node then we can use the put command to copy the file from the edge node’s local to hdfs.
(hadoop fs -put <local path> <hdfs path> 
or
 (hadoop fs -copyFromLocal <local> <hdfs>

Whenever a file gets stored in hdfs, it stores the file in the form of blocks which are distributed in clusters. For example, If a file is of 1.4gb (1408mb) then it will be divided in 11 blocks, 128 mb each along with its default replication factor.

2.	Component of Hadoop
	
Following are the components that collectively form a Hadoop ecosystem:
●	HDFS: Hadoop Distributed File System
●	YARN: Yet Another Resource Negotiator
●	MapReduce: Programming based Data Processing
●	Spark: In-Memory data processing
●	PIG, HIVE: Query based processing of data services
●	HBase: NoSQL Database
●	Mahout, Spark MLLib: Machine Learning algorithm libraries
●	Solar, Lucene: Searching and Indexing
●	Zookeeper: Managing cluster
●	Oozie: Job Scheduling
3.	Component of HDFS
The main components of HDFS are as described below:
NameNode is the master of the system. It maintains the name system (directories and files) and manages the blocks which are present on the DataNodes.
DataNodes are the slaves which are deployed on each machine and provide the actual stor­age. They are responsible for serving read and write requests for the clients.
Secondary NameNode is responsible for performing periodic checkpoints. In the event of NameNode failure, you can restart the NameNode using the checkpoint.

4.	Creating hive table on an unstructured file (serde properties)

5.	Map side join
 
6.	Partition 
7.	Bucketing 
8.	Find exception, error and other log from a unstructured file 

9.	ORC vs Parquet ? why ORC for Hive
10.	Broadcast join
11.	Function currying
12.	Data frame vs dataset ( why dataframe)
13.	Mysql query 

Id, name, subject , marks 
1 , abc, math , 50 
1 , abc, physics,60
1 , abc , chemistry, 20

Output-  abc, [math,physics, chemistry], 130

group_concat ( subject, separator ‘,’ )

2. Second highest selling product each day 
dense_rank()

Result - Got 
=========================surprise interview with legato================

Q select count(*) from EMP,DEPT;
	In the above query, what join will be applied?
Ans. 	Cartesian join.

Why?
As we haven’t mentioned the join keys, in all other joins (left, right, inner) we need to mention the key. As we haven’t mentioned it will do the cartesian join.
	Cartesian join?
	Ex:
	T1	T2
	C1 	C1
	1	23
	2	3
	4	1
	1	
	
	For every record in T1c1, it will match with all the record in T2C1
	So it will have a total of 12 records after executing the above query.

Q
select count(*) from EMP left outer join DEPT on EMP.deptno=DEPT.deptno;

Do it for left, right, inner
emp
10
20
30
40
50
60	dept
10
90
20


Ans. 7

Q
val a = "Hi Antriksh Kethwas"
o/p : ('H',1),('i',2),(' ',2)

Write code in scala to count each character out of string

========================= =========================
1.	Types of file format
2.	Can we load orc file format directly into hive table or have to use any other approach
3.	Write spark program to convert column into row
4.	How to eliminate the special character from file then load into hive table
5.	You have two table orders, a customer table. Find total amount for each month among the customer whose age is greater then 18
6.	Find the second highest salary from the emp table
7.	Find the common record from the two table without using inner join

========================= Dev impetus last round =========================
1.	How to create a partition  table in hive , write a syntax?

2.	How to load data in static  partition 
3.	Explain inheritance by creating classes to calculate area of shapes ( square, circle,rectangle etc).
4.	Explain word count program in map reduce phase.( input to the mapper , output of the mapper, input to the reducer and output to the reducer with internal working)
5.	Write a code to read data from hive and add a column based on the two columns if the value of both the column is same)
6.	How to send configuration files to all the nodes while submitting a spark job.
7.	Types of partition in spark
Ans.
	There are two types of partitions, hash partition and range partition. The choice on which range or hash partition will be applied depends on the following factors
a.	Available resources — Number of cores on which tasks can run on.
b.	External data sources — Size of local collections, Cassandra table or HDFS file determine number of partitions.
c.	Transformations used to derive -- RDD There are a number of rules to determine the number of partitions when a RDD is derived from another RDD.
C1 c2 c3
1 2 3
1 2 3

Write a query to get duplicate records ?

Select c1,c2,c3 from t1 group by c1,c2,c3 having count(*) >1;

Write a query to get the employee names those are not Manager using JOIN

=========================Antriksh MIQ  R2===============================
1.	Vendor of your Hadoop distribution /cluster ?
2.	How do dataframes code get internal optimization ??
3.	How does the catalyst optimizer work internally?
4.	Read a file in a data frame and count the number of shows watched and total views ?
 
Theboys avengers goodwillhunting Theboys
5.	Optimization you practice in spark jobs?


========================Antriksh Legato R2=================================
Question - 

tab A
 c1	tab B
c1
1
2
3
4
5	
1
2
3


Write a sql query to select only rows in tableA which are not matching in tableB. In above case, it is 4 & 5.

Ans.
select * from 
	A left join B
	on A.c1=B.c1
	where A.c1 not in (select * from B);


Q In an employee table, find the 5th highest salary.

Q 5th highest sal in employee table ?

Q (Scenario based questions on project in my organization)

Q How do you insert new partition data in a hive table?
Result - got 
===================extra questions==========================
1.How can we avoid/manage or clear hive_staging_date directories when we create or insert a table in hive ?
2. Which is faster beeline or hive shell ?
3. How does hive know that new data is added ? Does it have a mechanism to check data updates from a file on which the table is built ?
Drop an external table along with data
CREATE EXTERNAL TABLE IF NOT EXISTS names_text(
  a INT, b STRING)
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ','
  STORED AS TEXTFILE
  LOCATION '/user/andrena'
  TBLPROPERTIES ('external.table.purge'='true');    

ALTER TABLE addresses_text SET TBLPROPERTIES ('external.table.purge'='false');
  
=======================To The New - Antriksh============================

Q add a new column “grand total ” in the emp table against each employee record which will show the grand total of the department of that employee. You have input as below table:

table emp
empId empName sal grandTotal depId 
1		   a	35
2		   b	6
34		   a	35
4		   b	6

My approach:
grandtotal of dep

val depDf = empDf.groupBy(col("depId")).agg(expr("sum(sal)").as("grandTot").select(col("depId"),col("grandTot")


depDf2.join(broadcast(empDf), depDf2.depId===empDf.depId, left)

Q input = "baloon", write a function to return list of characters repeated more than once.

val input = "baloonbl"

def charCount(inp: String) = {
    
     inp.groupBy(x=>x).map(x=>(x._1,x._2.size)).filter(x=>x._2>=2).keys
}
charCount(input).foreach(println)

Q .  
empId, sal
12	400	
13	233	
14	111	

select empId,sal, rank() over (partition by dept order by sal desc) as salRank from empTable where salRank=2;

=====================Devs capgemini ==================================
I gave answer to all the technical questions
But he was not convinced regarding project or work i do,
Asked about => 
you not a part of deployment team
Deployment tool used to deploy the project 
Data pipeline 
File format you used 

Important == guy in the panel was MC with no manners 
====================================================================
Hi!!!!!
Please also add the interviewer's name while writing interview experience.
=====================================================================
Apple: First round interview for Devendra Kumar with Santhana Kumar Meikandan on 27th July 2021 @
=====================================================================
1- introduction 
2- detailed description about work
3- project in details , your contribution, challenges , other 
4- let assume we have a job running daily against the data we collect daily. And the job    usually takes a max of 4 hour to complete. one certain day we received the data (same size) and we ran the job and it took around 7 hours. 
What could be the issue ?How would you approach optimizing it?
5- Given an array of integer numbers and an integer target, return indices of the two numbers such that they add up to the target.
Input: nums = [3,2,4], target = 6
[2,3,3,4,8,9] // nlogn +  n 
//O(nlogn)
//1 = 
start ,end 
==
7
Output: [1,2]

object Solution {
    def twoSum(nums: Array[Int], target: Int): Array[Int] = {
        var ans = List[(Int,Int)] ()
        var tailed = nums.tail.toSet //n + (n-1) ------
        
        for( i <- nums ) // n
        {
            if(tailed.contains(target -i) //n
            {
               ans +: (nums.indexOf(i),nums.indexOf(target-i))
                             
              }
              tailed = tailed.tail
    }
    ans
}
//O(n^2)

6- in your job some tasks are taking a lot time and some of them are finishing their process in less time 
Explain possible issue  and solution 

===================================================================================================== Antriksh To the New round 2 ==================

1- Write a program to check whether a number is palindrome or not ?? using scala , without using predefined functions ?
2- write a query in mysql to select alternate records (you do not have any column with sequentially sorted data?
 
Select cl1, clo2,coln, row_number over(order by col1) rn where rn %2 ==0;

3- Can we allocate driver memory more than executor memory? If yes ? Explain such cases when you do the same?
Ans: And the driver-memory flag controls the amount of memory to allocate for a driver, which is 1GB by default and should be increased in case you call a collect() or take(N) action on a large RDD inside your application. Here 384 MB is the maximum memory (overhead) value that may be utilized by Spark when executing jobs.
4- you have 40 GB of executor memory , and the file size you want to process is around 80 GB. How would you process that?
80GB / 128MB 
Transformations +++

Some partition + tasks 
Scheduling tasks on the multiple executors 

5- what happens when the name node goes down in your cluster?
 Fsimage + editlog = name node restore
6- what happens when the data node goes down in your cluster? 
7- Mark down correct and incorrect statements ?

●	Select a,b,max(c) from t1  ()
●	Select a,b,max(c) from t1 group by a,b,c (correct)
●	Select a,b,max(c) from t1 group by a,b  (correct)
Select a,b,max(c) from t1 group by c  ()
A b c 
antriksh kethwas 40
antriksh kethwas 50
other name 40

antriksh kethwas 50
other name 40
============================================================================
APPLE :- Second round interview for Devendra Kumar with Abhishek Khandelwal on 5th August 2021 @ 4PM IST
============================================================================
1.	- introduction 
2.	- project and role / responsibilities 
3.	- we have a text file size 100 TB having -100 partitions, 1 tb each, what would be the best approach to check whether a given word exists in the file or not ??

4.	How dynamical allocation works in spark , how spark decides the number of executors ??  details ??
5.	What is the difference between sc.textFile and sc.wholeTextFile ??
6.	- given a file like 
	1
	2
	3
	.
	.
	1 tb data 

Val avg =  rdd.reduce(avg)

def avg(a: Int , b:Int ): Int ={

(a+b )/2
}

Tell me whether above code will compile or not ;

7.	Write a program to demonstrate the difference between map and flat map ? 
8.	What a tail recursion ?? write a program to reverse a string using tail recursion ?
object MyClass {
    def main(args: Array[String]) {
        //@tailrec 
        def rev (s:String) : String ={
           /*  
             if (s == "")
            s 
            else 
             rev(s.tail) + s.head 
             */
             s match {
              case ""  =>  return s
              case _  => rev(s.tail) + s.head 
              
             }
        }
        
        println( rev ("HELLO"))
    }
}

----------------------------------------------------------------------------------------------------------------------------------
The main difference, as you mentioned, is that textFile will return an RDD with each line as an element while wholeTextFiles returns a PairRDD with the key being the file path. If there is no need to separate the data depending on the file, simply use textFile.
When reading uncompressed files with textFile, it will split the data into chunks of 32MB. This is advantageous from a memory perspective. This also means that the ordering of the lines is lost, if the order should be preserved then wholeTextFiles should be used.
wholeTextFiles will read the complete content of a file at once, it won't be partially spilled to disk or partially garbage collected. Each file will be handled by one core and the data for each file will be one a single machine making it harder to distribute the load.
========================================================================
Table 1 - Workflow
id, name, wf_code
1, test wflof, "import re; import exp; var a = skfksf"
2, my abc wflow, "var key = my_passord"


Table 2 - wflow_sec_compliance
expression_type, expression_value
Slack, [xolb][a-b][0-9]
JDBC Pwd, [a-zA-Z][24]


sec_compliance
id, wf_name, wf_code, is_sec_cmp, matched_string
2, my abc wflow, "var pass", "1","password"

========================================================================
//val df1 = spark.sql("select * from workflow")
//val df2 = spark.sql("select * from wflow_sec_compliance")

def is_sec_cmp(str: String, reg:String): Char = 
{
	val reg2 = reg.r	

	case reg:
		match str
}

register.udf("is_sec_cmp",is_sec_cmp(str: String, reg:String): Char)

val df3 = spark.sql("select * from workflow, wflow_sec_compliance")	//cartesian

//id, name, wf_code, expression_type, expression_value df3
//id, name,wf_code,is_sec_cmp,matched_string dff

val df4 = df3.withColumn("is_sec_cmp", selectExpr("is_sec_cmp(wf_code, expression_value)")

============= MIQ’s 5th round (Antriksh’s int with Rohit) =======================
Rohit Srivastava2:09 PM
Workflow
id, name, wf_code
1, test wflof, "import re; import exp; var a = skfksf"
Rohit Srivastava2:10 PM
2, my abc wflow, "var key = my_passord"
Table 2 - wflow_sec_compliance
expression_type, expression_value
Slack, [xolb][a-b][0-9]
JDBC Pwd, [a-zA-Z][24]
Rohit Srivastava2:12 PM
sec_compliance
id, wf_name, wf_code, is_sec_cmp, matched_string
2, my abc wflow, "var pass", "1","passowrd"
You2:34 PM
Table 1 - Workflow
id, name, wf_code
1, test wflof, "import re; import exp; var a = skfksf"

2, my abc wflow, "var key = my_passord"

Table 2 - wflow_sec_compliance
expression_type, expression_value
Slack, [xolb][a-b][0-9]
JDBC Pwd, [a-zA-Z][24]

sec_compliance
id, wf_name, wf_code, is_sec_cmp, matched_string
2, my abc wflow, "var pass", "1","passowrd"


     =======================================================================
Dev’s Cognizant
======================================================================
1- project and actual work, pipeline , data source ?
2- designing pattern you use in your project
3-  hive internal(managed) / external table
Can we store managed table’s data in other loc than warehouse dir ?
4- syntax to read data in spark 
  Spark sql command to to get data based on some conditions 
5- how do you decide executors / executor’s memory / cores 
  Spark submit syntax
6- few more scenario based questions 
7- répartition / coalesce
8- What is COB and role based optimizer 
Cost Based optimization basically it required admin privileges
9- replace null value to 0 while loading data in to hive 
We can use coalesce
10- dataframe vs dataset vs RDD

  object Main {
  def main(args: Array[String]): Unit = {
    println("Hello world!")
  }
}
Imagine you are building some sort of service that will be called by up to 1,000 client applications to get simple end-of-day stock price information (open, close, high, low). 
You may assume that you already have the data, and you can store it in any format you wish. 
How would you design the client-facing service that provides the information to client applications? 
You are responsible for the development, rollout, and ongoing monitoring and maintenance of the feed. 
Describe the different methods you considered and why you would recommend your approach. 
Your service can use any technologies you wish, and can distribute the information to the client applications in any mechanism you choose. 

INPUT
stock=value
//df -> store 1min 
/data_dump/167293474929.csv  - minute 10000 stocks
stock,value
TCS,189
INFY,990

9AM - 4PM
10000Stocks per min

//open close high low 10000stocks
//1000 clients 10000 stock request

{ stock1: { open: close: high: low:} }

folder ( per min a file ) --> spark streaming  -> (1 min ) (prev , curr_staate) ( initial open , running high , running low )- >

 (open,close ,high , low  )

Given two arrays of integers, compute the pair of values (one value in each array) with the smallest (non-negative) difference. Return the difference.

		A[] = {1, 3, 15, 11, 2}
		B[] = {23, 127, 235, 19, 8} 
(11,8) 3

n^2 

var min_value = Integer.max
var ele1 = A(0); var ele2 = B(0)
for( i <- A){
for( j <- B)

 if( (i-j).abs < min_value)
 {
    min_value = (i-j)
 }    
min_value
}


// 1,2,3,11,15
// 8,19,23,127,235

B.min = 8 

order_details
	customer_name 		product_name		quantity		t_price
	john			        apple			      6			      10
	john			        milk			      3			      5
	jobin			        milk			      5			      8
	john			        apple			      10			    18
	jobin			        apple			      12			    20

Find the total quantity and total price per customer and per product

select customer_name, product_name, sum(quantity), sum(t_price) from order_details group by customer_name , product_name ;

Find the customer with highest spending 
select customer_name , rank( total ) R  over ( cluster by customer_name order by total) from( select customer_name, sum(t_price) total from order_details group by customer_name ) ) t where R =1 ;
 
 john 33 
 jobin 28 
jonnny 45


Result - no response after HR round (higher CTC expectation) 
==================================================================
Dev’s KPMG 
==================================================================
1.	Singleton vs companion object 
2.	Case class 
3.	Recursion vs tail recursion // write a program to find nth fibonacci number using tail recursion 
4.	How to read a file in scala (core scala)
5.	Map vs flatmap  (explain with an example) 
6.	Syntax to use map function with a lambda/ anonymous function 
7.	Write a syntax to find duplicates row based on 2 columns using windows function on a data frame
No -response after interview
==================================================================
Dev’s Quantiphi
=====================================================================
1- lambda function in python and map function  
2-  WAP to implement this - 
input = ["a", "z", "a", "c", "z"]
output -> {"a": true, "z": true, "c": false}  ( True if it appears twice/thrice, False if not )
def check ( list : List[Char]) ={

   var mapped = list.groupBy(identity).map(x => (x._1, x._2.size) .map( x => (x._1, if (x._2 >=2) true ; else false) 
  //               	      a,[a,a]		a->2			a->true
  //		     z,[z,z]		z->2			z-> true	
  //		     c, [c]			c->1			c-> false
  mapped 

}
# mapped = list.groupBy( lambda x : x) .map( lambda y :(y[0],y[1].len)) .map( lambda z : (z[0], 'true' if (z[1] >=2 ) else 'false'))

#x => x 

3- Tuple Vs List in python
4- functionalities of airflow  // how can we pass data between 2 taks 
5- kafka use cases 
6-  SQL query to find 4th highest salary
name , salary
7- SQL query to find running average of temp by month
year , month, day , temp
8- order by vs sort by 
9- partitioning / bucketing use cases
10- RDD use cases ? how does it achieve fault tolerance ?
11- cloud services / aws / GCP
Result - got 
==================================================================
===============Genpact for Morgan Stanley - Antriksh ================

Q Spark engine vs mapReduce engine

==========================================================
Dev Lumic = > BarRaiser
==========================================================
1.	Cloud knowledge/Big data tools
2.	Git
3.	Coding/Scripting
4.	Data Modeling/Database knowledge
Linux - commands( word coung , find vs grep) , concepts , architecture
Spark - scenario based , job running , concepts, data frame -coding, 
Hive -
HDFS -
Cloud - > EMR,glue, redshift , node failure, 
		How to import data from RDBMS into S3
Cron scheduler -   * * * * * * * what are 7 * in cron
Scala -  code to find largest substring having no duplicate chars
Git - revert last commit , git parse , 
Shell script -
SQL: 
Team1 Team2 MatchResult 
 RR KKR 2 
 MI CSK 2 
 RCB KXP 1 
 DD RR 0 
 KKR RR 1 
 CSK RCB 2 
 KXP DD 2

 Match Result descriptions: 
 1 => Match won by Team 1 
 2 => Match won by Team 2 
 0 => Draw

Output should have following columns

Team Played Won Lost Draw 
RR 3 0 2 1 
CSK 2 1 1 0 
RCB 2 2 0 0

Q. Length of the longest substring without repeating characters
Sample Input: "aabbcd"
Output: "bcd"
Result - I declined second round invitation 
====================================================================
Devs Altimetrik 
====================================================================
Round 1 => basic and famous questions, hive spark, scala, sql 

Round 2 =>  spark concepts , scala, sql queries
Keep the updated records + no old ad new extra records left 
Input1: old 
Name Phone Address
Jack xxxx BLR
John yyyy CH
Santosh xxxx HYD

Input2: updated 
Name Phone Address
Jack xxxx ND
John yyyy MA
Cherukuri bbbb Del

Output:
Name Phone Address
Jack xxxx ND
John yyyy MA
Santosh xxxx HYD
Cherukuri bbbb Del

Round 3 - EL , manager 
Result - not processing further due to higher salary expectation
=======================================================================
Neeraj’s PWC Tech-R1 
=======================================================================
(1)	You have been given (a,b,a,c,f,b,e,d) characters you have to put repeated characters in file1 and characters with exact count in file2.

val input = sc.textfile()

val rdd1 = input.flatmap(seperatedBy(","));
val rdd2 = rdd1. map (x => (x,1))
val rdd3 = rdd2 .reduceByKey((x,y) =>(x+y))

val rdd4 = rdd3.  

(a,1)
(b,2)

(2)	write a query for the product which has no transaction from the below table details. 


product -> pid, name, price
user -> uid, name, email
trxn -> trxid, uid,pid, date

select p.pid, p.name from product p left  outer join trxn t on 
p.pid = t.tid where t.txid=null;
					(or)
select * from product where pid not in(select distinct pid from trxn);

(3)	Difference between groupByKey() and reduceByKey().
(4)	Brief Discussion on the project.

=======================================================================
Neeraj’s PWC Tech-R2 
=======================================================================

(1)	Details Discussion on the project. What Challenges I have faced till now in the project and how to overcome them and what is your project pipeline.

(2)	You have been given below two dataframes  write your code -

DF1:User
Cols: UserId, Username

DF2: Events
cols: EventId, EventType, UserId, Timestamp

1.Display the count of events for each user
DF: Username, CountofEvents
=======================================================================
Neeraj’s Morgan Stanley Tech-R1 
=======================================================================
(1) All optimization techniques in spark.
(2) replica Factor in 
(3) All joins in SQL
(4) what do u mean by stages in Spark
(5) how to read csv file describe
(6) what is the advantages to store the data in hdfs with compare other files

(7) table-Result
	dept   dept_name 	Std_name 	std_total_mark 
 Q- Find out the student details who is having 2nd highest marks for each department.

(8) Do you know shell scripting?
(9) Describe a little bit about your project.


(10)
table A
col1
1
1
null

table B
Col1
1
1
1
null

inner join- 6 
left outer join - 7
right outer - 7
full cross - 12

(11)
table A
col1
1
1

table B
Col1
1
1
1

inner join-  
left outer join - 
right outer - 
full cross - 
=====================================================================
Dev’s fragma data
====================================================================
1- onLine test 
A-    Pyspark project ( implement 5 functions , reading data, filtering , joining , and writing back to HDFS use partition by a column) 
B- Spark questions - multiple choice 
C- Hive Questions -multiple choice
Video interview 
1- project 
2- some concepts of spark
3- sql normalization
4- ask about a table - never heard that table name
5- cloud - no experience 
6- databricks -  no experience 

====================================================================
Dev’s IQVIA 1st R
====================================================================

7- database vs data warehouse
8- spark internal , what happens when we run a spark job
Final round on - 29th sept

 SPARK -SCALA- 
1.    Syntax to create SparkSession? What is local in. master(“local”) 
2.    Data frame dataset
3.    Case class (use cases), syntax to create case class?
4.    Chase vs persist? How does memory and disk work?
5.    Different write modes in spark?
Ans : Save Modes/Write Modes
Save operations can optionally take a SaveMode, that specifies how to handle existing data if present. It is important to realize that these save modes do not utilize any locking and are not atomic. Additionally, when performing an Overwrite, the data will be deleted before writing out the new data.
Scala/Java	Any Language	Meaning
SaveMode.ErrorIfExists (default)	"error" or "errorifexists" (default)	When saving a DataFrame to a data source, if data already exists, an exception is expected to be thrown.
SaveMode.Append	"append"	When saving a DataFrame to a data source, if data/table already exists, contents of the DataFrame are expected to be appended to existing data.
SaveMode.Overwrite	"overwrite"	Overwrite mode means that when saving a DataFrame to a data source, if data/table already exists, existing data is expected to be overwritten by the contents of the DataFrame.
SaveMode.Ignore	"ignore"	Ignore mode means that when saving a DataFrame to a data source, if data already exists, the save operation is expected not to save the contents of the DataFrame and not to change the existing data. This is similar to a CREATE TABLE IF NOT EXISTS in SQL.

6.    What are all the transformations you have used in your project?
7.    Wide vs narrow transformations
8.    Traits??
9.    Diamond problem?
10. Higher order functions 
11. World count program in spark
12. What is Set in Scala?
13. What is a map Partition?
14. What is a map in Scala?
15. val states = Map("AL" -> "Alabama", "AK" -> "Alaska", "AZ" -> "Arizona")
find the value of “AL”
val ans = states("AL")
16. What is a static variable in Scala? Var vs Val
17. What error does it throw when you try to reassign a value in a static variable?
18. Error vs exceptions
19. What optimization techniques do you use?
20. Reading parquet file in spark, syntax
HIVE-
1.    Creating external table, syntax
2.    Creating partitioned table
3.    Partition and bucketing
4.    Loading data into Hive table
5.    Adding a partition in a hive table  
 Ans:  ALTER TABLE test
                     ADD PARTITION (dt=’2014-03-05’)
                     LOCATION ‘s3://test.com/2014-03-05’
6.    Removing duplicates from hive tables
7.    How to delete data with with schema of external table  
SQL-
1.    Nth highest salary from a table department wise
=======================================================================
Dev’s IQVIA 2nd R
====================================================================
1.	Write a word count program 
2.	Explain higher order function with an example // write code 
3.	Show latest / updated record 
EmpID EMPNM City Date
1 Jon Goa 201811
1 Jon Delhi 201901
2 Sonu Kolkatta 201901
2 Sonu Delhi 201709

select tmp.EmpID ,tmp.EMPNM , tmp.City from ( select EmpID ,EMPNM , City, dense_rank( ) over ( order by date  desc) rnk from Employee ) as tmp where rnk = 1 ;

Show employee details with all the cities he has visited => 
select EmpID, EMPNM , group_concat(City) from Employee ;/// ( group by EmpID,EMPNM)
4.	Spark / concepts 
5.	Spark submit commands 
> spark-submit --class class1 spark_job.jar --deploy-mode client /cluster  --master yarn --conf   --executor-core 5 --executor-memory 18G --num-executors 20 //--max-executors 
--driver-memory 50G 

6.	Add a column before given column
7.	Data frame vs Data set 
8.	
Result :- HSE ….

======================================================================
DEv’s Tekion 1st R coding 
========================================================================
1.	Given n sorted linked lists , merge them in sorted manner?
1 -> 2 -> 3 
3 -> 4 -> 5
5 -> 9 -> 10
…….
Output - 1 -> 2 -> 3 -> 3 -> 4 -> 5 -> 5 -> 9 -> 10 

2.	Given a list of Strings , combine all the anagrams together in List of lists?
input: strs = ["eat","tea","tan","ate","nat","bat"]
Output: [["bat"],["nat","tan"],["ate","eat","tea"]]

def groupAnagrams( strs):
      result = {}
      for i in strs:
         x = "".join(sorted(i))
         if x in result:
            result[x].append(i)
         else:
            result[x] = [i]
      return list(result.values())

print(groupAnagrams(["eat", "tea", "tan", "ate", "nat", "bat"]))


Rejected 

================================================================
DEv’s EY 
===================================================================
Telephonic => 20 mins 
Spark,Scala,Hive   concepts

Video interview => 
Spark internal working 
Scenario based question on  partition after  left join  on 2 dfs having 10 and 20 partitions 
Dag , how it works 
stages , scenario based on transformation , how many stages 
Code on Dataframe => return all the columns  as a list of string
    withcolum() with match case 

=======================================================================
Neeraj’s Quantiphi Assessment round 
=======================================================================
-	There will be 20 MCQ questions based on sql, hive, spark, scala some coding input/output
 	
=======================================================================
Neeraj’s Quantiphi Tech-R1 
=======================================================================
 You have been given below 2 tables

Table: Orders

Column                  Type
OrderID                 Integer
CustomerID         Integer
Purchase_date   Timestamp
ProductID    	 Integer
Product                String
Quantity              Integer
Price         	Float

Table: Customers

Column                  Type
ID                     	   Integer
name                      String
Contact_no           Integer

Q1-Query to find all the customer names who have purchased the top 5 selling products in the Year 2018.
	
/* select distinct customers.name from orders join customers on orders.cid=customers.id where productid in 
	(select t1.productid from (
	select productid, sum(quantity) as sale from orders where year(purchase_date)='2021' group by productid order by sale desc limit 3) t1
    ) ;
  by antriksh*/



Ans - Suppose data is available in below format. 

101 123 todaysdate 100 TV 3 Rs90000
102 234 todaysdate 100 TV 2 Rs60000
103 123 todaysdate 200 SamusungMobile 1 Rs 50000
104 234 todays date 200 SamusungMobile 2   Rs100000

select (select from dense_rank()over(order by price desc)prc)temp where temp.prc=1 and date between

Q2 - You have been given below table. Write a sql query to delete the duplicate records

CREATE TABLE sales_team_emails (
   id INT AUTO_INCREMENT,
   sales_person_name VARCHAR(255),
   sales_person_email VARCHAR(255),
   PRIMARY KEY (id)
);

Create table unique_records as (Select id,sales_person_name, sales_person_email from sales_team_emails group by id,sales_person_name, sales_person_email); solution by Antriksh

Insert overwrite table sales_team_emails (select distinct * from sales_team_emails); 
in hive - solution by dev

Example: 
+----+-------------------+----------------------+--------------------------------
| id | sales_person_name | sales_person_email   	       |
+----+-------------------+----------------------+--------------------------------|
|  1 | Aditi 	   	 |  aditi@hotmail.com 	        |
|  2 | James F Stone 	 |  jamesstone@yahoo.com   |
|  3 | James Stone   	 |  jamesstone@yahoo.com   |
|  4 | Aditi Sharma	 |  aditi@hotmail.com              |
+----+-------------------+----------------------+----------------------------------

Partial Ans(Which I tried)
with EMPYEECTE As (select *, row_number() over (partition by id order by id) as row_number from sales_team_emails ) 
delete from EMPYEECTE where row_number >1

Q3- Write a hive query to view all the databases whose name begins with “db”
	Show databases like db “db*”;
Q4- Write word count program in spark.
Q5- What is a higher order function in scala.
Q6- What is a wide and narrow transformation?

=======================================================================
Neeraj’s Quantiphi Managerial Round 
=======================================================================
●	Discussion regarding your project, issues which you have faced in your project.
●	How will you handle your junior’s if they work under you? How is your interaction with the client?


=======================================================================
Neeraj’s CitiusTech-R1
=======================================================================
Q1 - What do you mean by decorators ?
Q2 - You have been given below table
eName		 eSalary 	eDepartment
Sat 		5555 		HR
mat 		6767 		IT

Now you have to add new column ‘current_date’ 

eName		 eSalary 	eDepartment 		current_date
Sat 		5555 		HR 			09-04-2021    
mat 		6767 		IT 			09-04-2021   

Suggestion -  Here you can use withColumn() to add a new column. 

Q3 - Write Sql query to find out 2nd highest salary of each department.
Ans- select tmp.emapname,tmp.sal,tmp.dept (select empname,sal,dept dens_rank() over(group by dept            order by sal desc)rank) tmp where tmp.rank=2;

Q4- You have been given two table- Table1 & Table2
table1
ID
1
1
null
0
0
3

table2
ID
0
0
1
null
4
6
7

-	What will be the total record  for inner join.(Ans -6) 

==============================================================
Neeraj’s CitiusTech-R2 (Managerial)
=============================================================
●	Brief discussion about your project, what is your project flow ?
●	No. of team members in your project, What is your role ?
●	How you inject spark with hive

==========================================================
Neeraj’s Scienaptic-R1
=============================================================
Q1- You have been given a table with 2 columns A & B. Replace all ‘0’ with ‘*’

table   				table  
A    B				A    B
1    0				1    *	
0    1		=>	               *    1
0    0				*     *
1    1				1    1

Partial Ans-
val rdd1 = rdd.flatmap(x=> x.split(" "));
val rdd2 =  rdd1.map(x=> x.split(" ")(0),x.split(" ")(1)) 
val rdd3 = rdd2.filter(x => if(x._2 = 0) {(x._1,*)})

Q2- You have a csv format file with 1000 columns and 1cr of row. How will you optimize it at the time of reading and processing ?
Q3- What do you mean by rolling window, tumbling window, sliding window ?
Q4- When can we get Out Of Memory in the master node 
Q5- Difference between groupByKey() and reduceByKey().

==================================================================
Neeraj’s Altimetrik-R1
=================================================================

Q1- Given a table containing bus route details, write a query/program to find all possible routes between two stations and time between them.

Input :

Busid 	Station 	Arrival_Time
1 	st1 		4:20
1 	st2 		5:30
1 	st3 		7:30

Output:

Busid 	Route 		Duration
1 	st1-st2 		70 
1	st2-st3 		120
1 	st1-st3 		190

Q2- what is your project pipeline?
Q3- which mode you use in your project client mode or cluster mode and what is the reason to use that mode?
Q4- How to debug jobs ?
Q5- What optimization do you use in your project ?
====================================================================
Naman’s Legato
======================================================================

Q1. Difference between managed and external tables in hive?
Q2. coalesce vs repartition ?
Q3. What is DAG?
Q4. What is a trait?
Q5. Some questions related to the project, where to take your data and where to dump your data ?
Q6. Write a query to find the nth highest salary?
Q7. Which model are you using agile and waterfall?
Q8. What is left equi join?
Q9. Difference between reducebykey and groupbykey?
Q10. Transformations, wide transformation vs narrow transformation?
Q11. Higher order functions?
Q12. Which technique did you use for join optimization?

=====================================================================
					Naman’s Impetus
=======================================================================

Q1. Hive Architecture?
Q2. Spark Architecture?
Q3. Interface in scala?
Q4. Some questions related to project flow?
Q5. Client mode vs Cluster mode?
Q6. What about executor,core,ram?

=======================================================================
					Naman’s Tiger Analytics
========================================================================
Q1. Input :- [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
       Output :- out1 - [1,5,9,13]
          Out2 - [2,6,10,14]
         Out3 - [3,7,11,15]
         Out4 : [4,8,12,16]

Q2. Input :- [math - 40, physics-50, chemistry - 80, hindi-90, english - 70]
	The marks of the subject is greater than 60 then print all the subjects?
       Output :- [name: vishnu {chemistry,hindi,english}]  

Q3 There will be a table cust_id,sales_order,quantity,time. So you write a query who will be the max sales of this table?

Q4: Write a word count problem using rdd?

Q5. There are some data like :-
1.	 john cena
2.	 rohit sharma
3.	chris gayle
You can write a program using dataframe/dataset and change the first lower letter to upper letter
Output :- 
1. John 
2. Rohit Sharma
3. Chris Gayle  

=================================================================					Naman’s Capgemini
=================================================================
Q1. Which optimization technique is used in spark?
1) Broadcast join
2) Cache/Persist as and when needed
3) Filtering unnecessary data before doing joins
4) Apart from 1,2 and 3, we use delta format which supports data skipping/optimize/z-order
5) Enabled Adaptive Query Execution at cluster level.
6) resource level optimization
7) use kryo serializer
Q2. Managed table vs external table?
Q3. Write a program to find a number that will be odd or even?
Q4. Write a query to find the max salary?
Q5. write a query to show the 5 names whose salary will be highest?
Q6. Difference between map vs flatmap?
Q7. How many transformations are used in a project?
Q8. What are shared variables?
Q9. What is case class?

==========================================================
					Naman’s Coforge
==========================================================
Q1. Write a command to edit a file in linux?
Q2. Write a command to validate a file in linux?
Q3. Difference between  external table and managed table?
Q4. Write a query to find the 6th highest salary?
Q5. Write a word count program in rdd?
Q6. Difference between spark context and spark session ?
Q7. Class vs case class?
Q8. What is a higher order function ?
Q9.  Differ between repetition and coalesce?

========================Bhavesh - Publicis sapient=================

1. Spark Context vs Spark Session?
2. How Many Spark Context are Allowed in one Spark Session?
3. Repartitioning vs Coalesce?
4. Partitioning vs Bucketing?
5. Sql query for second highest salary?
6. Which version of spark are you using?
7. What is case class, how is it different from Normal class and Object in scala?
8. What happens internally when you do spark-submit?
9. What is broadcast join?
10. Can you run different spark jobs in the same cluster with different configurations?

==========================Bhavesh - Cognizant ====================

1. Project Overview?
2. Given Spark cluster with 5 executors with 5 cores for each, how many tasks can run in parallel?
3. Explain Spark Architecture?
4. Word count program?
5. Find the Highest salary for each department?

=========================Bhavesh - Reliance Jio====================

1. What are your day-to-day responsibilities in the current project?
2. Explain about your project Architecture?
3. Transactional vs Non-Transactional database?
4. Managed table vs External Table?
5. Singleton object vs singleton class?
6. Window functions LEAD() vs LAG()?
7. Dataset vs RDD?
8. Partitioning vs Bucketing?
9. Given a XML file with Name as a Column like: divide the Name column into two columns with Name    First_Name and Last_Name and store the result in another table?

                        |        Name             |
                    _________________________
                   |    Bhavesh Patidar       |
                   |    Ashutosh Singh        |
                   |    Naman Sharma         |
                   |    Neeraj Jaiswal          |


========================= Bhavesh - Legato ===============================

1.	How Data can be transferred from one cluster to another cluster?
2.	If we perform partitioning on particular column values and we get a NULL value for that column, will it throw any Error?
3.	Repartitioning VS Coalesce?
4.	What does the MSCK repair command do?
5.	Can we change the location of the Managed Table?
6.	Why is the Replication factor in Hadoop is 3 only?
7.	What is cost Based Optimization in Spark?
8.	If we perform join on two DF, what will be default RDD Partition Internally?
9.	Linux Command create three Directories A,B,C inside a temp Directory?

======================Bhavesh - Numeric Technologies ======================

1.	How do you read data from s3 buckets and create Hive tables, write commands?

create external table table_tmp (.....)
row format delimited fields terminated by '',"
location 's3://...//.....//'

2.	What is the default table in Hive?
Managed table

3.	Can we change the location of the managed table, write commands to change the location of the managed table, and where data is stored in the case of the managed table?

Yes, you can do it by using the clause – LOCATION ‘<hdfs_path>’ we can change the default location of a managed table.

4.	What are the configuration files in Hadoop?
Configuration Files are the files which are located in the extracted tar.gz file in the etc/hadoop/ directory.
All Configuration Files in Hadoop are listed below,

1) HADOOP-ENV.sh->> It specifies the environment variables that affect the JDK used by Hadoop Daemon (bin/hadoop). We know that the Hadoop framework is written in Java and uses JRE so one of the environment variables in Hadoop Daemons is $Java_Home in Hadoop-env.sh.

2) CORE-SITE.XML->> It is one of the important configuration files which is required for runtime environment settings of a Hadoop cluster.It informs Hadoop daemons where the NAMENODE runs in the cluster. It also informs the Name Node as to which IP and ports it should bind.

3) HDFS-SITE.XML->> It is one of the important configuration files which is required for runtime environment settings of a Hadoop. It contains the configuration settings for NAMENODE, DATANODE, SECONDARYNODE. It is used to specify default block replication. The actual number of replications can also be specified when the file is created,

4) MAPRED-SITE.XML->> It is one of the important configuration files which is required for runtime environment settings of a Hadoop. It contains the configuration settings for MapReduce . In this file, we specify a framework name for MapReduce, by setting the MapReduce.framework.name.

5) Masters->> It is used to determine the master Nodes in a Hadoop cluster. It will inform about the location of SECONDARY NAMENODE to Hadoop Daemon.
The Mater File on Slave node is blank.

6) Slave->> It is used to determine the slave Nodes in Hadoop cluster.
The Slave file at Master Node contains a list of hosts, one per line.
The Slave file at Slave server contains IP address of Slave nodes.

5.	How will you install Hive on top of Hadoop?
Pata hai but Nahi bataunga , Admin nahi banana hume

6.	What is dynamic partitioning, what is the property to set dynamic partitioning true?

1.	create a non-partitioned table t2 and insert data into it.
2.	now create a table t1 partitioned on the intended column(say country).
3.	load data in t1 from t2 as below:
hive> INSERT INTO TABLE t2 PARTITION(country) SELECT * from T1;

In static partitioning we need to specify the partition column value in each and every LOAD statement.
suppose we are having partition on column country for table t1(userid, name,occupation, country), so each time we need to provide country value
hive>
LOAD DATA INPATH '/hdfs path of the file' INTO TABLE t1 PARTITION(country="US") 
hive>
LOAD DATA INPATH '/hdfs path of the file' INTO TABLE t1 PARTITION(country="UK")

7.	What is the default port No. for Hadoop?

8.	Difference b/w order by and sort by?
Hive supports SORT BY which sorts the data per reducer. The difference between "order by" and "sort by" is that the former guarantees total order in the output while the later only guarantees ordering of the rows within a reducer. If there are more than one reducer, "sort by" may give partially ordered final results.
Note: It may be confusing as to the difference between SORT BY alone of a single column and CLUSTER BY. The difference is that CLUSTER BY partitions by the field and SORT BY if there are multiple reducers partitions randomly in order to distribute data (and load) uniformly across the reducers.
Basically, the data in each reducer will be sorted according to the order that the user specified. The following example shows
SELECT key, value FROM src SORT BY key ASC, value DESC

9.	Given a table with column “Field1 '', write sql query to Create usernames from given mail id?
Ex:    
                |     Field1      |                                                  output
               |  —-------------  |
                   Swapnil.abc@yahooo.in                           Swapnil.abc
                  Bhavesh.patidar@google.com                 Bhavesh.patidar

(mysql)
select substring(t.field1,1,position("@" in field1)-1) from (select "antrikshkethwas@gmail.com" as field1) t;


10.	What is Vectorization in Hive?
Vectorization allows Hive to process a batch of rows together instead of processing one row at a time. Each batch is usually an array of primitive types. Operations are performed on the entire column vector, which improves the instruction pipelines and cache usage. HIVE-4160 has the design document for vectorization and tracks the implementation of many subtasks.

Enable Vectorization in Hive

To enable vectorization, set this configuration parameter:

hive.vectorized.execution.enabled=true 
When vectorization is enabled, Hive examines the query and the data to determine whether vectorization can be supported. If it cannot be supported, Hive will execute the query with vectorization turned off.

11.	Explain Traits in Scala?
Like abstract classes, they can not be instantiated and they can be either implemented or unimplemented. Traits are created so that they could be inherited by other classes or even other traits. Main property of a trait is that multiple traits can be inherited in a single class. Traits enable multiple inheritance in Scala. Pseudo code Example of multiple inheritance using trait is below:

Trait A{
	Var a
	Def fun
}
Trait B
{
	Var a = 30
	Def fun2:String{ “how are you” }
}

Class newcl extends A, B{}

Newcl class above can inherit from multiple traits which would have been not possible if trait A and trait B were classes instead of traits

12.	Case Class in Scala?
Case class A( name:String, id:int)
Case class B(name:String, id:int)
println(A.name)

	//A==B is possible


13.	How will you measure the performance of Spark Job?

Using spark web UI
Checking task execution time of each stage. The difference between longest execution task and shortest execution should be minimum for a spark job to be performant.


======================= Bhavesh - Coforge ============================

1.	Project Overview?
2.	Relational vs NoSql Database?

Relational Database	NoSQL
It is used to handle data coming in low velocity.	It is used to handle data coming in at high velocity.
It gives only read scalability.	It gives both read and write scalability.
It manages structured data.	It manages all types of data.
Data arrives from one or few locations.	Data arrives from many locations.
It supports complex transactions.	It supports simple transactions.
It has a single point of failure.	No single point of failure.
It handles data in less volume.	It handles data in high volume.
Transactions written in one location.	Transactions written in many locations.
Deployed in vertical fashion.	Deployed in Horizontal fashion.

3.	What are different serde techniques?
●	SerDe is a Library which is built-in to the Hadoop API
●	Hive uses Files systems like HDFS or any other storage (FTP) to store data, data here is in the form of tables (which has rows and columns).
●	SerDe - Serializer, Deserializer instructs hive on how to process a record (Row). Hive enables semi-structured (XML, Email, etc) or unstructured records (Audio, Video, etc) to be processed also. For Example If you have 1000 GB worth of RSS Feeds (RSS XMLs). You can ingest those to a location in HDFS. You would need to write a custom SerDe based on your XML structure so that Hive knows how to load XML files to Hive tables or other way around.

4.	Cache vs Persist?
5.	What are the different lateral views?
-----------------+---------------------+--+
| employee.ename  | employee.dept_list  |
+-----------------+---------------------+--+
| Tom             | [20]               	 |
| Jerry           | [10,20]      	 |
| Riley           | [20,30,40]              |
SELECT ename, dept_id FROM employee LATERAL VIEW explode(dept_list) depts AS dept_id;

6.	Traits in Scala?
In scala, trait is a collection of abstract and non-abstract methods. You can create traits that can have all abstract methods or some abstract and some non-abstract methods. A variable that is declared either by using val or var keyword in a trait gets internally implemented in the class that implements the trait.
 
======================= Antriksh - Deloitte ==============================

1st part: explain your projects
2nd part:questions on project:	
1.	scheduling tools being used?
a.	What operators you used in airflow
2.	ETL pipeline in your project?
	3rd part: Spark and sql questions
		
Input table
cid	cname	ctype
1	abc	std
2	def	prem
3	hij	all

Output table
cid	cname	ctype
1	abc	std
2	def	prem
3	hij	std
3	hij	prem

Write sql query to get above output from given input table.
Write Spark code to get above output using given input dataframe


Sql: (temporary solution)
select d2.id,d2.name,d1.ctype
	from delloite d1
    join delloite d2
    on d1.id!=d2.id where d1.ctype!="all";



Spark:
val data = List((1, "abc", "std"),
        (2, "def", "prem"),
        (3, "hij", "all"),
        (4, "klm", "std"),
        (5, "opq", "all")
       )
val rdd = sc.parallelize(data)

//devendra
import org.apache.spark.sql.functions._

val rdd2 = rdd.map(x=>if(x._3=="all"){ (x._1,x._2,("std prem")) } else x)
val df3 = rdd2.toDF("cid","cname","ctype")
val df4 = df3.withColumn("ctype2", explode(split($"ctype", " "))).drop("ctype")

df4.show()

================= dev coforge =====================
1- hive meta store 
2- external/managed 

Question - 1 
test1_a table : ID-> 10,20,30,40
test1_b table : ID -> 10,30,50
write a code to fetch values in test1_a that are not in test1_b without using NOT or MINUS keyword/operator?
desired output : (20,40)

val l1 = List(10,20,30,40)
val l2 = List(10,30,50)
 
val df1 = sc.parallelize(l1).toDF("id")
val df2 = sc.parallelize(l2).toDF("id2")
val df3 = df1.join(df2,df1("id")===df2("id2"),"left")
val df4 = df3.filter(df3("id2").isNull).drop("id2")
df3.show()
df4.show()

Question -2 
The table Employee has 3 columns(EmpID, OfficeMobile, HomeMobile). 
Some Employees have given the same number for both places. write a code where output
should be 2 columns only EmpID and Contact No. 
If number is same, put in 1 row else in 2 rows.
 
Sample :
EmpID, OfficeMobile,HomeMobile
1,   	 123,   	 123
2,   	 456,   	 789
desired output :
1,123
2,456
3,789
 
 
import org.apache.spark.sql.functions._
val tb = List((1,123,123),(2,456,789))
val df = sc.parallelize(tb).toDF("EmpID","OfficeMobile","HomeMobile")
 
val df2 = df.withColumn("phone",when(col("OfficeMobile")===col("HomeMobile"), col("OfficeMobile"))
                             .otherwise(concat(col("OfficeMobile"),lit(" "),col("HomeMobile")))
                       ).drop("OfficeMobile","HomeMobile")
val df3 = df2.withColumn("contact", explode(   split(col("phone"), " ")   )   ).drop("phone")
 
// val df2 = df.withColumn("new_gender", when(col("gender") === "M","Male")
//       .when(col("gender") === "F","Female")
//       .otherwise("Unknown"))
 
df3.show()

======================================================================

Q- What happens in the background when we fire spark-submit considering a job submitted in yarn cluster mode, right from sending the request to cluster manager and returning the output to destination.
Answer = > When Spark submit is invoked with deploy mode as cluster, it will choose a node randomly to launch the driver. Once the driver is launched , Driver will take the help of Resource Manager(YARN in this case) to decide where to launch the executors considering availability of resources on node.
Once executors gets spawned up , driver will send all of the artifacts ( which are specified in the spark-submit , it will be main artifact i.e. Jar/Py file..

Then Spark Engine follows the lineage (Logical Plan) which will be converted to DAG(Physical Plan) during creation of stages. There could be multiple jobs (Each action corresponds to a job in spark) in a spark application. Each Shuffle operation will create a new stage, and inside the stage we will have multiple tasks (which is the smallest entity) and spark keeps working on each task(equivalent to partition). This happens lazily when action gets invoked.
Once each action inside a spark application gets completed , the resources (Driver/executor) gets released.

=====================================================================

What to do if the processing data size is larger than the Spark memory size/ driver memory size or the spark cluster memory size?

And, How to process 1TB of data with 64GB of memory in Apache Spark? How does Spark work with it internally?

In Apache Spark, if the data does not fit into the memory then Spark simply persists that data to disk.
It depends on whether you need the full terabyte to be in memory at once or not. If your 1TB of data is actually 1 million 1MB records that can be processed independently, then no problem. All you need to do is load your data into an RDD and repartition it such that each partition is less than 64GB in size. This is because Spark will load each partition into memory during processing.

Spark is an In-Memory processing tool, but obviously, it can’t put everything in memory if the data is huge. So, what will it do?
Discard it? That wouldn’t be ideal. Nobody wants that. So, the only other option left is to store it back on disk.
And, so that is exactly what it does. Put it in the disk until it's needed. When the data is needed, it swaps it in, and so on.
But, remember that this process can be costly as we see with MapReduce. Mapreduce stores every result onto the disk and takes input from the disk. So, It is really slow. Even though Spark does fast computations in-memory, It won’t be really useful if you have to keep shuffling the data around. So, you just have to be careful in those cases.

Does my data need to fit in memory to use Spark?
No. Spark's operators spill data to disk if it does not fit in memory, allowing it to run well on any sized data. Likewise, cached datasets that do not fit in memory are either spilled to disk or recomputed on the fly when needed, as determined by the RDD's storage level.

A spark driver is a machine that obviously cannot hold TerraBytes' worth of data. Let's say you have a file or multiple files of 10TB and you want Spark to perform some computation of this file. The driver machine will just create a Logical plan on how to perform computation on this file, it will come up with the best possible and optimized plan (with the help of tungsten and catalyst optimizer), the file is partitioned and each partition is read by executors that are in machines closer to where the blocks exist (as per data locality in HDFS).
So spark can handle data size much bigger than the driver since the read operation doesn't happen at the driver. Certain actions like collect can overwhelm the driver since it collects all the executor results to the driver before writing down the result.


========================Antriksh’s LnT 2nd round========================

Q.size the small table when we broadcast? Both in spark and hive
Hive:25mb
Spark:10mb

Q. how many partitions does spark create when you read a file?
Ans.
As many number of blocks were there when the file was stored in HDFS, initial stage of our spark job will have same number of partitions 
(no. of partitions in stage0 = no. of blocks used to store file in hdfs)

Q. How is group by key different performance wise then reduced by key?

Q.what happens if you have a file of bigger size than the memory can accommodate, what would you use cache or persist and why?

Q. where would you use cache/persist in your code? Scenario when you used it?

Q. When and why does shuffling of data occur from partition to partition?

=============================Avinash Volkswegan 1st round=================
1)	Introduction
2)	what project you are done
3)	what is your task in daily
4)	how sqoop do execution internally
5)	what is boundary val query 
--split-by For free-form query imports, you need to specify 'split-by' . When you are importing the result of any particular query, sqoop needs to know the column-name using which it will create splits. Whereas, while importing tables, if not specified, it uses the primary key of the table being imported for creating splits. In case your primary key is uneven and not consistent, you can also specify any other column using split-by.
--boundary-query During sqoop import process, it uses this query to calculate the boundary for creating splits: select min(), max() from table_name.
In some cases this query is not the most optimal so you can specify any arbitrary query returning two numeric columns using --boundary-query argument. This saves min(split-by) and max(split-by) operations and thereby is more efficient.

6)	what if no primary key in table then how it will execute
7)	ideally how many mapper should be there in sqoop while fetch join tables data
8)	how would you deal delta load or data change in daily
9)	how can give secure password in scoop
10)	what are sorting algorithms
11)	what is spark (exact meaning of general purpose , in memory , compute engine)
12)	how spark it better than map reduce
13)	what is repartition and coalesce
14)	what is catalyst optimizer and what optimization it do internally
Catalyst optimizer
Optimizer (aka Catalyst Optimizer) is the base of logical query plan optimizers that defines the rule batches of logical optimizations (i.e. logical optimizations that are the rules that transform the query plan of a structured query to produce the optimized logical plan).

It consists of rules like
=>filter pushdown
=>combining of filters
=>combining of projections
=>we can also add our own rules.

15)	what is stages 
16)	why still we required low level rdd while having df and ds
17)	what is feature of spark 3 
●	2x performance improvement on TPC-DS over Spark 2.4, enabled by adaptive query execution, dynamic partition pruning and other optimizations
●	ANSI SQL compliance
●	Significant improvements in pandas APIs, including Python type hints and additional pandas UDFs
○	Better Python error handling, simplifying PySpark exceptions
○	New UI for structured streaming
○	Up to 40x speedups for calling R user-defined functions
○	Over 3,400 Jira tickets resolved
○	 

18)	what is feature of spark 2

19)	diff bw spark 2 and spark3 
Spark 3 has 
AQE(re-optimizing at each stage until last stage)
 spark3 has feature that it can dynamically coalesce shuffle partitions
It can dynamically switch join strategies, choosing the most optimized one
Dynamically optimizing skew joins

20)	how to search a word in the file , write a program
21)	how to fetch data from sqoop with out server down time
22)	how to fetch data from s3 with the sqoop import
23)	Complexity for binary search algorithm


Result : Not selected : 

I  got very good comments from interviewer but consultancy person told your name is not shortlisted

=============================Avinash PWC 1st round=================
1.	q: write sqoop import command
sqoop import \
--connect jdbc:mysql://localhost/userdb \
--username root \
--table emp_add \
--m 1 \
--target-dir /queryresult

2.	q:how to handle delta load in sqoop
(1) Incremental Append
(2) Incremental Last Modified
3.	q:what is kind of file format(please tell me each in detail)
4.	q:what is partition and bucketing and when do we use what
5.	q:how much data you are reading into the spark
6.	q:what kind of complex data type we have in hive
7.	q: write a word count program (how can you make this more dynamic)
8.	q: how to you submit spark program , can you tell me end to end flow
9.	q: Can you tell me how to submit your jar file and once it is submitted what we do exactly ?
10.	q: what is a cluster machine and client machine ?
11.	q: can you write the data frame structure from parquet file format ?
12.	q: What is the lineage graph in spark ?
13.	q: what is difference between persistent and cache()
14.	q: Can you explain persistent storage levels ?
15.	Q: what is default partition in spark


ABC     XYZ
id	num
1	1
2	1
3	1
4	2
5	1
6	2
7	2

consecutiveNums
1
Q: exactly needs to check for numbers appearing at least three times consecutively
Q:tbl
               
SELECT DISTINCT
       num
       FROM (SELECT num,
                    lag(num, 1) OVER (ORDER BY id) = num
                    AND lag(num, 2) OVER (ORDER BY id) = num c3
                    FROM logs) x
       WHERE c3;


name	dept	salary
james	sales	3000
michael	sales	4600
robert	sales	4100
maria	finance	3000
james	sales	3000
scott	finance	3300
jen	finance	3900
jeff	marketing	3000
kumar	marketing	2000
saif	sales	4100

find the name of employees earning more than the average salary of their department ?
Tell me about the last 5 records here  ?
 
  select e.first_name, e.salary, e.department_id
    from employees as e
    where e.salary > (select avg(e2.salary)
                  from employees e2
                  where e2.department_id = e.department_id
                 );

Result : Pending

I explained each theory concept well but got stuck in queries and interview time duration exact 1 hour with Aditi Singh


=============================Avinash TAVANT 1st round=================
1.	q: how to migrate data from hdfs to s3
2.	q: what is surrogate key
3.	q: what is kafka streaming
4.	q: what is star schema & snowflake schema and which scenario use what
5.	q: what is primary key & foregin key
6.	q: what is diff between unique key and primary key
7.	q: what is emr & ec2
8.	q: how to load data in parques file
9.	q: what is azure ?
10.	q:why not Gcp your company is using ?
11.	q:tell me how google search internally works?
12.	q: what is AWS Glue and how do you use it in Etl activity?
13.	q:what is athena and how amazon takes charge from customers on running queries?

Resule : Not Selected with Tanmay

Tavnat is looking for a Kafka & cloud expert candidate.

=============================Avinash Genpact 1st round=================

Q: How to fetch data with sqoop
Q: what is normalize and denormalize
Q: how did you break a table into normalize form
Q: what is LIT in data frame
Q: what is partitioning
Q: HOW would you load data manually in partition table
Q: what is data frame
Q: how did u join two table in data frame
Q: what is bucketing
Q: how your project architecture
Q: what is athena and aws glue
Q: how to normalize a table (can you do it right now )
Q: how good are you in sql ?
Q: tell me how many joins are there and diff between full outer and union 
Q: What exploded ?
Q: Why do you use salting ?
Q: Which file format did you use ?
Q: Can you write your code for word count ?

Result : Selected 1st round 

There were two panel members and one was observing you full time and another was asking only questions


==========================Antriksh’s Cognizant=============================


Q) When you submit your spark application, what cluster size do you give and why?

Q) hadoop cluster vs spark execution cluster
	
Q) generic questions like repartition coalesce, cache persist……..


=========================Antriksh Impetus second round-=======================

Q you have a variable string "aaaaabbccaa"
o/p should be 5a 2b 2c 2a
Write the code in any programming language

val str = "aaaaabbccaa"

var cnt = 0
var elem = str(0)
var ans = ""

for(i <- str)
{
    if(i==elem)
    {
        cnt=cnt+1
    }
    else
    {
        ans=ans+elem+cnt
        elem=i
        cnt=1
    }   
}
ans=ans+elem+cnt
println(ans)

Q) you have a file with each line having 40 characters, you need to write a code to create a new df with 4 columns. We do not have any delimiters in data. Each column has 5,10, 10,15 characters respectively.

Ans.

40 characters

1)read

val rdd = spark.sparkContext("path")

rdd.map(x=>	{
	(substr(x._1,0,5)),(substr(x._1,5,15)),(substr(x._1,15,25)),(substr(x._1,25,40))	)
}
)
	

Q)

table
id,name,	add1,	add2
1	ant		 ind	 ujj
2	sunil	null	noida
3	anil	delhi	null

trgt tbl
id,	name,	address
1	ant		 ind
1	ant		 ujj
2	sunil	noida
3	anil	delhi


var df = spark.read....(input table)

//df.withColumn("address",when(df(col("add1").isNull and 

var df2 = df
df.join(df2,df(id)===df2(id), "inner")

id,name,	add1,	add2
1	ant		 ind	 ujj
1	ant		 ind	 ujj
2	sunil	null	noida
2	sunil	null	noida
3	anil	delhi	null
3	anil	delhi	null

df.withColumn("address",


========================= DEV PAYTM TEST ==================================
select  v.state, group_concat(v.votes) from 
	( select t.state, concat(t.name, " x " , count(t.candidate_id) ) as votes , count(t.candidate_id) as cnt from 
    (
    select concat(c.first_name," " ,c.last_name) as name,r.state,r.candidate_id from candidates c 
	join results r on c.id = r. candidate_id 
    ) t
     group by t.state,t.name  order by count(t.candidate_id) desc 
     ) v group by v.state; 
	 
	 
candidate 

id  	first_name		 last_name
1	avi 			porval
2 	dev 			kumar 


result 

cand_id   	state 

1			UP
2			MP	
1			UK
1			BH
2			PJ
3			
4
5
1
6


output - 

state 	 result 

UP 		avi porwal X 3, dev kumar X 2 
MP      		ant kethwas X 10, avi porwal X 3 
--------------------------------------------------------------------------

input1 = 5
input2 = 6 
numbers = 7


[5,6,55,56,66,555,556,566,666]

------------------------------------------------------------------------

===========================Antriksh’s Reliance jio 1st round===========================



Q) Write a program to write all those records in a file that have at least one column value as null and the rest of the records in other file or table

schema
aadhar, cname, cstate, cage, network_performance, timestamp

my approach: 

//df = spark.read(path).option("mode","dropmalformed") interviewer doesn't want it like this

df_all = spark.read()...

var df_missing = df_all.select(when(col("aadhar")==null),or(col("cname")=null)

var df_everythingelse = df_all.filter(df_all(col("aadhar") == null or df_all(

df_everythingelse.write.parquet("path")


Q)on the given schema, what columns would you use for bucketing and partitioning. explain with reason

schema
aadhar, cname, cstate, cage, network_performance, timestamp

my ans:
cstate and cage as partitioning columns
timestamp or aadhar as bucketing column

due to cardinality of cstate and cage is low, we can use them as partitioning columns
and since cadrinality of timestamp//aadhar is low, we can use anyone as bucketing column


Q)

WITH T(StyleID, ID)
     AS (SELECT 1,1 UNION ALL SELECT 1,1 UNION ALL SELECT 1,1 UNION ALL SELECT 1,2) T
SELECT *,
       RANK() OVER(PARTITION BY StyleID ORDER BY ID)       AS 'RANK',
       ROW_NUMBER() OVER(PARTITION BY StyleID ORDER BY ID) AS 'ROW_NUMBER',
       DENSE_RANK() OVER(PARTITION BY StyleID ORDER BY ID) AS 'DENSE_RANK'FROM   T;

stleid, id 
1, 1
1, 1
1, 1
1, 2

Ans:

stleid, id, rank, row_num, dense_rank
1,	 1, 	1,	 1,		 1
1,	 1, 	1,	 2,		 1
1,	 1, 	1,	 3, 		 1
1,	 2, 	4,	 4, 		 2





















In [0]:
Q there are 5 ids (key) , 10 partitions in data. How many files will be there?
ans.	50 files
 val ddf2 = ddf1.repartition(10)
ddf2.write.format("csv").option("path", "partitioned_data2").partitionBy("sid").save
ee
Q Now I want 5 files only (as many number of keys are there)
ans. coalesce()
val ddf2 = ddf1.coalesce(1)
ddf2.write.format("csv").option("path", "partitioned_data3").partitionBy("sid").save


In [0]:
data = [1,2,3,4,5]
schema =["id"]

df = spark.createDataFrame(data,schema)
df.show()

ddf2 = df.repartition(10)
ddf2.write.format("csv").partitionBy("id").option("path", "/Volumes/bigdata/assign/assign_file").save()

In [0]:
data = [1, 2, 3, 4, 5]
schema = ["id"]

df = spark.createDataFrame(data, schema)
display(df)

ddf2 = df.repartition(10)
ddf2.write.partitionBy("id").format("delta").mode("overwrite").option(
    "path", "/Volumes/bigdata/assign/assign_file").save()

In [0]:
data = [1, 2, 3, 4, 5]
schema = ["id"]

df = spark.createDataFrame(data, schema)
display(df)

# Write to a managed Delta table in Unity Catalog
df.write.partitionBy('id').mode("overwrite").save("/Volumes/bigdata/assign/assign_file/part_test")

In [0]:
%sql
DESCRIBE HISTORY delta.`/Volumes/bigdata/assign/assign_file`

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RepartitionWrite").getOrCreate()

# Sample data with 5 keys
data = [
    ("C001", "UP"), ("C002", "MP"), ("C003", "MH"),
    ("C004", "UP"), ("C005", "MP"), ("C006", "MH"),
    ("C007", "UP"), ("C008", "MP"), ("C009", "MH"),
    ("C010", "UP")
]
df = spark.createDataFrame(data, ["customer_id", "sid"])

# Repartition to 10
df_repartitioned = df.repartition(10)

# Write with partitionBy → creates 5 folders × 10 files = ~50 files
df_repartitioned.write \
    .format("csv") \
    .mode("overwrite") \
    .partitionBy("sid") \
    .save("/Volumes/bigdata/assign/assign_file/part_test.csv")